---
# Covid-19 Analysis
---
Gruppo 3:

- Alessio Contin matr. 734792
- Stefano Santini matr. 726396

Laboratorio di Data Science

---
# Table of contents
1. [Introduzione](#introduction)
2. [Province](#province)
3. [Regioni](#regioni)
    1. [Analisi e visualizzazione](#analysis)
    2. [PCA](#pca)
    3. [LAG](#lag)
    4. [Autocorrelation e Partial Autocorrelation](#autocorrelation)
    5. [Time Series](#ts)
    6. [Clustering](#clustering)
        1. [K-Means](#kmeans)
        2. [DBSCAN](#dbscan)
    7. [Regressione](#regression)
    8. [Forecasting](#forecasting)
    9. [Modello logistico](#logistic)
    10. [Modello SIR](#sir)
    11. [Modello SEIR](#seir)
    12. [Modello SEIR con distanziamento sociale](#seird)
    13. [SARIMAX](#arima)
    14. [Altri modelli per forecasting](#otherforecasting)
4. [References](#references)

Le domande a cui vorremmo dare risposta sono le seguenti:
1. quali sono le province più colpite in base al numero di contagi?
2. quali sono le regioni più colpite? (valutazione di cosa si intende con maggiormente colpita sulla base dei vari indici epidemiologici)
3. analizzare l'andamento nel tempo delle diverse features per ogni regione
4. forecasting per valutare l'andamento delle features, ad esempio quando il numero di nuovi contagiati sarà sotto una certa soglia.


---
## Introduzione <a name="introduction"></a>

Inizialmente importiamo tutte le librerie necessarie all'esecuzione dei successivi codici.

In [ ]:
# Importazione delle librerie
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
import datetime as dt
import plotly.express as px

%matplotlib inline
%run -i "ts.py"

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits import mplot3d

from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import metrics
from scipy import interpolate

#from fbprophet import Prophet
#from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py


I dataset impiegati sono quelli di Kaggle prelevati in data 23/03/2020, relativi all'Italia, uniti ad un dataset creato, sfruttando le infomrazioni presenti nei seguenti siti:

Attributo realtivo alla popolazione--> https://www.tuttitalia.it/regioni/popolazione/

Attributo relativo ai posti in terapia intensiva--> http://www.quotidianosanita.it/studi-e-analisi/articolo.php?articolo_id=82888

---
## Province <a name="province"></a>

Importiamo il dataset relativo alle province ed effettuiamo le prime analisi e visualizzazioni relative ad esso.

In [ ]:
#carico il dataset 
province_italy=pd.read_csv('covid19_italy_province.csv')

In [ ]:
#visualizziamo informazioni sulle colonne e sui loro tipi
province_italy.info()

In [ ]:
#visualizziamo come è formato il dataset come numero di righe e colonne
province_italy.shape

Per avere una prima impressione sul contenuto del dataset visualizziamo alcune righe.

In [ ]:
#visualizziamo le prime cinque righe
province_italy.head()

In [ ]:
#visualizziamo le ultime cinque righe
province_italy.tail()

Da una prima esplorazione del dataset, tramite le funzioni head() e tail(), notiamo che vi sono alcuni missing values, codificati come NaN nella colonna ProvinceAbbreviation, e come 'In fase di definizione/aggiornamento)' nella colonna ProvinceName.

Per avere più informazioni proviamo ad invocare la funzione describe() 

In [ ]:
province_italy.ProvinceName.describe()

Invocando la funzione describe sulla feature ProvinceName notiamo che ha 108 valori univoci, ovvero i nomi delle 107 province, più il valore che consideriamo come nullo 'In fase di definizione/aggiornamento'

Utilizziamo la funzione describe anche sull'altra feature che contiene valori nulli, ovvero 'Province Abbreviation

In [ ]:
province_italy.ProvinceAbbreviation.describe()

Notiamo che vi sono 106 valori univoci, invece di 107, numero delle province. Indaghiamo ulteriormente visualizzando la parte di dataset che ha ProvinceAbbreviation nullo.

In [ ]:
province_italy[province_italy.ProvinceAbbreviation.isnull()]

Notiamo nella riga 5 che abbiamo l'attributo ProvinceName che ha come valore Napoli, mentre la rispettiva ProvinceAbbreviation risulta nulla. Questo perchè pandas ritiene che NA non sia l'abbreviazione di Napoli ma l'indicazione di un valore nullo. 

Gestiamo il problema dei valori nulli e dell'abbreviazione di Napoli.

In [ ]:
#temp=province_italy
mask = province_italy.ProvinceName == 'Napoli'
column_name = 'ProvinceAbbreviation'
province_italy.loc[mask, column_name] = 'NAP'

Mostriamo come il dataset modificato adesso nella colonna relativa all'abbreviazione di Napoli non mostri più valore nullo ma il valore NAP

In [ ]:
province_italy[province_italy.ProvinceAbbreviation == 'NAP'].head()

In [ ]:
#Prima di eliminare i nulli dal dataframe originario, li copiamo su un
#dataset temporaneo, nel caso ci servissero per qualche analisi successiva
temp=province_italy[province_italy['ProvinceAbbreviation'].isnull()]

Abbiamo deciso di rimuovere i valori nulli, in quanto i relativi valori di TotalPositiveCases variano molto, indicando che i valori che erano in fase di aggiornamento, in parte nei giorni successivi vengono attribuiti alle rispettive province.

In [ ]:
#Rimuoviamo i valori nulli
province_italy.dropna(inplace=True)

In [ ]:
#Verifichiamo che non ci siano più valori nulli
province_italy[province_italy.ProvinceAbbreviation.isnull()].count()

Abbiamo verificato che non ci siano più valori nulli, adesso vediamo le prime informazioni sul numero totale di positivi

In [ ]:
province_italy.TotalPositiveCases.describe()

Per visualizzare meglio i dati, memorizziamo in un dataframe provvisorio i dati relativi all'ultima giornata presente, in modo da visualizzare il totale dei positivi corrente

In [ ]:
currentDf=province_italy[province_italy.Date == province_italy.Date.max()]

Ordiniamo il dataframe per il totale dei casi positivi, in ordine decrescente, in modo da selezionare la top 10 delle province per numero di contagiati attuali.

In [ ]:
#ordiniamo il dataframe
topDf=currentDf.sort_values(by='TotalPositiveCases', ascending=False)

In [ ]:
#Selezioniamo la top10 delle province
topDf=topDf.iloc[:11,:]

In [ ]:
#visualizziamo la top 10 delle province
topDf.head(10)

Per visualizzare meglio i dati, li proponiamo in formato visivo attraverso l'uso di grafici

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.barplot('ProvinceAbbreviation', 'TotalPositiveCases', data=topDf)

Dalla visualizzazione e analisi precedente notiamo che la maggior parte dei casi sono in Lombardia,Piemonte e Veneto, con alcuni anche in Marche ed Emilia Romagna. Per dare una visione globale, in rapporto anche alle province di altre regioni, diamo una visualizzazione grafica su una mappa interattiva.

In [ ]:
#Visualizziamo su una mappa, utilizzando la libreria plotly
fig = px.scatter_geo(currentDf, lat='Latitude', lon='Longitude',
                     hover_name="ProvinceAbbreviation", size="TotalPositiveCases",
                     projection="natural earth", scope='europe', color="TotalPositiveCases", center={'lat':41.902782, 'lon':12.496366})
fig.show()

Dal dataframe precedente, contenente le top 10 province all'ultima data disponibile, ricaviamo le loro abbreviazioni 

In [ ]:
provinces=[]
for i in topDf.ProvinceAbbreviation:
    provinces.append(i)
print(provinces)

Utilizzando le abbreviazioni, partendo dal dataframe originale, otteniamo un nuovo dataframe contenente le informazioni relative a ciascuna delle province presenti nella top 10 per ogni giorno presente nel dataset.

In [ ]:
#nuovo dataframe per top 10 province con arco temporale completo
topDfOverTime=province_italy[province_italy.ProvinceAbbreviation.isin(provinces)]

Mostriamo tramite grafici l'andamento del numero totale positivi nel corso dell'intero arco temporale presente per le province maggiormente colpite.

In [ ]:
#grafico andamento
fig, ax = plt.subplots(figsize=(20,10))
topDfOverTime.groupby(['Date','ProvinceAbbreviation'])['TotalPositiveCases'].max().unstack().plot(ax=ax, marker= 'o')

Visualizziamo delle boxplot.

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.boxplot('ProvinceAbbreviation', 'TotalPositiveCases', data=topDfOverTime)
plt.xlabel('ProvinceAbbreviation') # Set text for the x axis
plt.ylabel('TotalPositiveCases')# Set text for y axis

Per dare una visione globale, in rapporto anche alle province di altre regioni nell'arco temporale completo, diamo una visualizzazione grafica su una mappa interattiva.

In [ ]:
#mappa interattiva che mostra diffusione durante tutto arco temporale
fig = px.scatter_geo(province_italy, lat='Latitude', lon='Longitude', 
                     hover_name="ProvinceAbbreviation", size="TotalPositiveCases", animation_frame="Date",
                     projection="natural earth", scope='europe', color="TotalPositiveCases", center={'lat':41.902782, 'lon':12.496366})
fig.show()

---
Diamo un'occhiata all'andamento della provincia più colpita, ovvero Bergamo.

In [ ]:
#creiamo un dataframe con i dati relativi a Bergamo
BG_df=province_italy[province_italy.ProvinceAbbreviation == 'BG']

Per capire meglio l'andamento del contagio, aggiungiamo una colonna che indica il numero del giorno dall'inizio dell'epidemia.

In [ ]:
dateCount=province_italy[province_italy.ProvinceAbbreviation == 'BG'].Date.count()
ticks=range(0,dateCount,1)
BG_df=BG_df.sort_values(by='Date', ascending=True)
BG_df['DayNumber']=ticks
BG_df.head()

Visualizziamo l'andamento tramite grafici.

In [ ]:
#province_italy[province_italy.ProvinceAbbreviation == 'BG']
fig, ax = plt.subplots(figsize=(20,10))
g=sns.barplot('DayNumber', 'TotalPositiveCases', data=BG_df)

In [ ]:
#sns.lineplot(data=prova, palette="tab10", linewidth=2.5)
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot('DayNumber', 'TotalPositiveCases', data=BG_df, marker='o')

Notiamo che l'andamento segue una curva molto accentuata, quasi esponenziale, con un numero di contagi superiore anche a città più grandi come ad esempio Milano. Resta da capire perchè il numero di contagi si è concentrato in Lombardia, se c'è stato qualche evento particolare, o se c'è qualche relazione anche in base a parametri e risultati che vedremo nella sezione sulle regioni, analizzando il relativo dataset, che contiene dati aggiuntivi come il numero di decessi, di ospedalizzati, di positivi e nuovi positivi, nonchè il numero di tamponi e di posti letto in terapia intensiva. 

---
## Regioni <a name="regioni"></a>

---
## Analisi e Visualizzazioni <a name="analysis"></a>

Come nella sezione precedente, importiamo i dataset effettuiamo le prime analisi e visualizzazioni.

In [ ]:
#caricamento dataset
regione_orig = pd.read_csv('covid19_italy_region.csv',parse_dates=['Date'])
popolazioni_orig=pd.read_csv('popolazioni.csv')

In [ ]:
regione = regione_orig.copy() 
popolazioni=popolazioni_orig.copy()

Dopo aver copiato i dataset di interesse, si esegue un 'merge' in base al nome della regione.
Successivamente si visualizzano le dimensioni e le informazioni sul datset creato.

In [ ]:
regione2 = pd.merge(regione, popolazioni, left_on="RegionName", right_on="Regione").drop('Regione', axis=1)

In [ ]:
regione2.shape

In [ ]:
regione2.info()

In [ ]:
#visualizziamo primi cinque elementi
regione2.head()

In [ ]:
#visualizziamo ultimi cinque elementi
regione2.tail()

Sono stati creati nuovi attributi per meglio analizzare la situazione delle diverse regioni sotto più punti di vista.
Di seguito viene proposta una spiegazione di ogni nuova feature.

------------------------------------------------------------------------------------------------------
LEGENDA

Se a t(2) ho CurrentPositive=100 questo è uguale a NewPositive(t(0))+NewPositive(t(1))

"TotalHospitalizedPatient" + "HomeConfinement" = "CurrentPositive"

"IntensiveCare" + "HospitalizedPatient" = "TotalHospitalizedPatients"

"CurrentPositive" + "Deaths" + "Recovered" = "TotalPositiveCases"

Viene evidenziata una dipendenza tra i vari attributi.


SPIEGAZIONE VARIABILI

ratio_PIR_Pop: corrisponde al rapporto tra i posti letto in terapia intensiva a regime con tuatta la popolazione (entrambi i parametri sono relativi alla regione).

ratio_IC_HP: corrisponde al rapporto tra i pazienti in terapia intensiva e quelli ospedalizzati (entrambi i parametri sono relativi alla regione).

ratio_TPC_TeP: rappresenta il rapporto tra i casi positivi totali e il numero di test (tamponi) eseguiti sulla popolazione (entrambi i parametri sono relativi alla regione).

ratio_TPC_Pop: definito anche come prevalenza, rappresenta il numero di casi positivi totali rispetto alla popolazione (entrambi i parametri sono relativi alla regione).

ratio_D_Pop: definito anche come tasso di mortalità, corrisponde ai decessi sul totale della popolazione (entrambi i parametri sono relativi alla regione).

ratio_TCP_Pop_gg: definito anche come tasso di incidenza grezzo, definisce il numero di casi in un dato intervallo temporale (entrambi i parametri sono relativi alla regione).

proportion_IC_HP_TPC: proporzione tra tutti i pazienti in ospedale e il numero di casi positivi totali (entrambi i parametri sono relativi alla regione).

proportion_HC_TPC: proporzione tra il numero di paziento ospedalizzati e il totale delle persone che hanno la patologia (entrambi i parametri sono relativi alla regione).

proportion_CPC_TPC: proporzione tra totale dei pazienti in ospedale sommato a quelli confinati a casa e il numero di casi positivi totali (entrambi i parametri sono relativi alla regione).

proportion_R_TPC: proporzione tra il numero di guariti e il numero di casi positivi totali (entrambi i parametri sono relativi alla regione).

proportion_IC_PIR: definita anche come indice di occupazione, definisce il numero di posti letto occupati, in questo caso in terapia intensiva (entrambi i parametri sono relativi alla regione).

proportion_D_TPC: definita anche come letalità, definisce il numero di decessi sul totale della popolazione (entrambi i parametri sono relativi alla regione). 

---------------------------------------------------------------------------------------------------------------

Per la creazione del tasso di incidenza è stato implementato il seguente codice in modo da avere per ogni regione e per ogni numero di giorni trascorso la rispettiva incidenza che rappresenta la frequenza di contagi.
Es: il valore ottenuto dal rapporto verrà diviso per il numero di giorni trascorsi: al primo giorno avrò una divisione per 1, al secondo per 2, al 28 per 28.

In [ ]:
#Tasso di Incidenza x 100000 abitanti
regione2.insert(regione2.shape[-1], 'ratio_TCP_Pop_gg', (regione2['TotalPositiveCases']/regione2['Popolazione'])*100000)

In [ ]:
regione2

In [ ]:
# Valutazione dell'incidenza relativa ad ogni regione per oggni giorno
a=0
b=int(regione2.shape[0]/21)
i=1

while b<regione2.shape[0]+1:  
    i=1
    for k in range(a,b):
        rowIndex = regione2.index[k]
        regione2.loc[rowIndex, 'ratio_TCP_Pop_gg'] = (regione2.loc[rowIndex, 'ratio_TCP_Pop_gg'])/i
        i=i+1
        
    a=a+int(regione2.shape[0]/21)
    b=b+int(regione2.shape[0]/21)
    

In [ ]:
regione2.head(84)

In [ ]:
regione2.tail()

Gli indici creati vengono moltiplicati per i fattori 100, 10000 a seconda dei casi, come da prassi epidemiologica, al fine di ottenere valori in percentuale o rapportati ad un dato quantitativo di abitanti.

In [ ]:
#Inserisco nella tabella creata alcuni nuovi attributi di interesse e alcune misure di incidenza epidemiologica
   
#Valore di posti Intensivi x 10000
regione2.insert(regione2.shape[-1], 'ratio_PIR_Pop', (regione2['PostiIntensivaRegime']/regione2['Popolazione'])*10000)
#Valori percentuali
regione2.insert(regione2.shape[-1],'ratio_IC_HP', (regione2['IntensiveCarePatients']/regione2['HospitalizedPatients'])*100)
regione2.insert(regione2.shape[-1],'ratio_TPC_TeP', (regione2['TotalPositiveCases']/regione2['TestsPerformed'])*100)
#Prevalenza x 10000 abitanti
regione2.insert(regione2.shape[-1], 'ratio_TPC_Pop', (regione2['TotalPositiveCases']/regione2['Popolazione'])*10000)
#Mortalità x 10000 abitanti
regione2.insert(regione2.shape[-1], 'ratio_D_Pop', (regione2['Deaths']/regione2['Popolazione'])*10000)


In [ ]:
#Valori percentuali
regione2.insert(regione2.shape[-1],'proportion_IC_HP_TPC', ((regione2['HospitalizedPatients']+regione2['IntensiveCarePatients'])/regione2['TotalPositiveCases'])*100)
regione2.insert(regione2.shape[-1],'proportion_HC_TPC', (regione2['HomeConfinement']/regione2['TotalPositiveCases'])*100)
regione2.insert(regione2.shape[-1],'proportion_CPC_TPC', (regione2['CurrentPositiveCases']/regione2['TotalPositiveCases'])*100)
regione2.insert(regione2.shape[-1],'proportion_R_TPC', (regione2['Recovered']/regione2['TotalPositiveCases'])*100)
# %Occupazione
regione2.insert(regione2.shape[-1],'proportion_IC_PIR', (regione2['IntensiveCarePatients']/regione2['PostiIntensivaRegime'])*100)
# %Letalità 
regione2.insert(regione2.shape[-1],'proportion_D_TPC', (regione2['Deaths']/regione2['TotalPositiveCases'])*100)




In [ ]:
#Visulaizziamo Le informazioni relative al dataset creato, per valutare se vi sono valori NaN
regione2.info()

Si osserva la presenza di valori nulli; questi verranno considerati in seguito. 

Per ora si considerano i dati relativi all'ultima data nota.

In [ ]:
#Seleziono gli elementi degli attributi relativi all'ultimo giorno, dato che ogni elemento è cumulativo 
regione_new = regione2[regione2["Date"] == max(regione2["Date"])].reset_index()

In [ ]:
max(regione2["Date"])
regione_new

Si prosegue con la creazione di un dataframe con i soli attributi di interesse relativi all'ultima data e per ogni regione.

In [ ]:
#Raggruppo per regione e seleziono gli attributi di interesse
regione_new2=regione_new.groupby('RegionName')['HospitalizedPatients','CurrentPositiveCases', 'Deaths', 'IntensiveCarePatients', 'Recovered', 
                                               'HomeConfinement', 'TotalPositiveCases', 'TestsPerformed', 
                                               'Popolazione', 'PostiIntensivaRegime','ratio_PIR_Pop', 'ratio_IC_HP',
                                               'ratio_TPC_TeP','ratio_TPC_Pop','ratio_D_Pop','ratio_TCP_Pop_gg',
                                               'proportion_IC_HP_TPC','proportion_HC_TPC','proportion_CPC_TPC',
                                               'proportion_R_TPC','proportion_IC_PIR','proportion_D_TPC'].max()


In [ ]:
#Stampo a video il dataframe
regione_new2

In [ ]:
#Stampo a video i dati di ogni regione, sfruttando una funzione di Pandas per evidenziare i dati sulla base della 
#loro rilevanza all'interno della colonna
regione_new2.style.background_gradient()

La tabella sopra proposta rappresenta un utile strumento per valutare l'incidenza della malattia da diversi punti di vista.
Come si osserva la Lombardia è la regione che possiede i valori più alti per quasi tutti gli indici.
Considerando gli indici ordinari, cioè quelli che non sono stati derivati (quelli epidemiologici), si osserva che le regioni più interessate sono Lombardia, Emilia Romagna, Veneto e Piemonte.
Tuttavia questi indici tengono in considerazione un solo aspetto e sono piuttosto rigidi.
Andiamo ad osservare cosa emerge dagli indici derivati:

ratio_PIR_Pop --> Emilia Romagna e Valle d'Aosta primeggiano per il numero di posti letto in terapia intensiva per 10000, decisamente inferiore rispetto a quella della Lombardia, nonostante abbiano una popolazione inferiore a quest'ultima. La Lombardia si attesta ad un livello simile a quello delle regioni centro-meridionali. 

ratio_IC_HP	--> in questo caso le regioni che hanno un percentuale maggiore di pazienti in terapia intensiva, rispetto a quelli in ospedale, sono la Basilicata e la Campania. Quindi queste due regioni hanno più pazienti in terapia intensiva di quelli in ospedale nonostante la popolazione ed il numero di contagiati siano inferiori rispetto alla Lombardia, Veneto, Piemonte.

ratio_TPC_TeP --> Lombardia, Marche e Valle d'Aosta emergono per aver il maggior numero di contagi rispetto al numero di tamponi effettuati. Tuttavia non è del tutto vero che all'aumentare del numero di tamponi effettuati in una regione, il numero di contagiati aumenta, in quanto il Veneto è al secondo posto come numero di tamponi eseguiti, ma di questi poco meno del 9% sono positivi.

ratio_TPC_Pop --> Valle d'Aosta, Lombardia, Marche e P.A. Trento hanno i valori più alti di contagi rispetto alla popolazione.
Questo porterebbe a pensare che minore è la popolazione maggiore sia il numero di contagi, quindi la rapidità risulta maggiore; tuttavia maggiore è la popolazione, maggiore è il numero di soggetti suscettibili al contagio. Per valutare questo aspetto è necessario osservare il Tasso di Incidenza.

ratio_D_Pop	--> La Lombardia ha il valore di morti per Covid x 10000 abitanti più alto. Ovviamente per quanto detto sopra maggiore è il bacino di soggetti influenzabili, maggiore è la probabilità di avere un numero alto di contagiati e quindi un alto numero di morti.

ratio_TCP_Pop_gg --> il Tasso di Incidenza evidenzia che Valle d'Aosta e Lombardia sono le più interessate e quindi le più colpite dal punto di vista della frequenza dei contagi. Questo indice mette in evidenza che, sulla base di quanto accennato sopra, sia la rapidità che il bacino di influenza sono aspetti egualmente importanti nella diffusione di una malattia.
Utile per realizzare strategie di controllo e risposta per la malattia.

proportion_IC_HP_TPC --> Piemonte, Lazio, Liguria, Molise e Abruzzo sono le più interessate dal punto di vista dei pazienti che richiedono trattamenti ospedalieri per la malattia, rispetto al numero di contagiati.
Questo fa pensare che qualora il numero di contagiati non sia eccessivo alcune regioni applichino di più la politica di cura ospedaliera, salvo il fatto dei trattamenti di terapia intensiva che di per sè sono necessari, compatibilmente alla capacità del loro sistema sanitario.

proportion_HC_TPC --> Valle d'Aosta, Sardegna e Basilicata primeggiano per il numero di paziento positivi confinati a casa, ma si osserva che la maggior parte delle regioni ha valori elevati per questo indice. Comparando i valori con quelli dell'indice precedente emerge il fatto che le regioni con valori inferiori, relativi all'indice in esame, sono quelle che hanno un maggior numero di pazienti che necessitano di cure ospedaliere.

proportion_CPC_TPC --> La Lombardia ha la percentuale minore di pazienti confinati a casa e ospedalizzati rispetto alla totalità dei casi positivi. Ciò significa che la restante percentuale, più alta rispetto alle altre regioni è composta da soggetti guariti e deceduti.

proportion_R_TPC --> La componente percentuale complementare a quella del punto precedente risulta, nel caso della Lombardia, composta per lo più dai soggetti guariti.
Nei casi più evidenti quali Emilia Romagna e Molise, si osserva che tale componente percentuale è per la maggior parte composta da soggetti deceduti.

proportion_IC_PIR --> Lombardia, Marche e Valle d'Aosta hanno la più alta saturazione dei posti di terapia intensiva rispetto al numero di contagiati. Questo parametro è di sicuro molto importante per valutare l'impatto della malattia sulle disponibilità delle risorse intensive di ogni regione. Quindi non solo le regioni più popolose hanno criticità nella terapia intensiva, ma anche quelle con un numero di abitanti inferiori. Potrebbe essere uno spunto per valutare politiche di riadeguamento regionale delle risorse in caso di situazioni critiche come quella attuale.

proportion_D_TPC --> La Lombardia, Emilia Romagna e Liguria hanno alti valori, tuttavia vale quanto riportato in 'proportion_R_TPC'

Sulla base di quanto osservato emerge che i parametri di maggior valore sono proportion_IC_PIR e ratio_TCP_Pop_gg, pertanto le regioni maggiormente interessate dalla malattia sono Lombardia e Valle d'Aosta. Tuttavia gli altri indici restano validi per comprendere le varie sfaccettature del problema.

Creazione di un nuovo dataframe per osservare i dati globali dell'Italia, relativamente alle features ordinarie.

In [ ]:
#Raggruppo per regione e seleziono gli attributi del dataset originale per ottenere una valutazione sull'italia
regione_new3=regione_new.groupby('Date')['HospitalizedPatients','CurrentPositiveCases', 'Deaths', 'IntensiveCarePatients', 'Recovered', 
                                         'HomeConfinement', 'TotalPositiveCases', 'TestsPerformed',
                                         'Popolazione', 'PostiIntensivaRegime'].sum()
           

In [ ]:
#Stampo i dati totali di tutte le regioni per avere una visione globale
regione_new3.style.background_gradient()

In [ ]:
#Realizzazione di un grafico delle dispersioni per ogni attributo
scatterplot_matrix  = pd.plotting.scatter_matrix(regione_new2, alpha=0.2, figsize=(15, 15), diagonal='kde', s=200)
for ax in scatterplot_matrix.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')
plt.show()

Lo scatterplot non permette un immediata valutazione delle relazioni tra le features derivate.
Si intuisce un rapporto di dipendenza per quanto riguarda quelle ordinarie.

Si esegue un grafico heatmap per valutare meglio se è presente una correlazione.

In [ ]:
#Dato che il grafico delle dispersioni è poco utile a intuire eventuali relazioni si effettua un matrice di correlazioni
sns.set(font_scale=1.4)
plt.figure(figsize=(20,15))
sns.heatmap(regione_new2.corr(),annot=True)
plt.axes().set_title('Heatmap n°1',fontsize =20)
plt.show()

Dalla matrice di correlazione si evidenzia il fatto che è presente un'effettiva dipendenza tra gli attributi appartenenti al dataset originale, uniti anche ai due attributi definiti come 'Popolazione' e 'PostiIntensivaRegime'. In questo contesto la correlazione è diretta. 
Per le misure di incidenza si evidenziano casi in cui si ha una correlazione diretta, ma anche casi in cui tale correlazione è indiretta, ciò è dovuto alla natura dell'operazione matematica di rapporto e proporzione. A tal proposito si nota la presenza anche di bassi valori di correlazione compresi tra circa -0.4 e 0.4.

Si procede alla creazione di variabili per poter eseguire una visualizzazione grafica di alcune features del dataset e per meglio comprendere gli andamenti di queste, come riportato nella tabella 'background'

In [ ]:
#Esecuzione di un sort dei dati all'interno di ogni attributo
pop= popolazioni.sort_values(by= ['Popolazione'], ascending=False)
positive_cases = regione_new.sort_values(by = ['CurrentPositiveCases'], ascending=False)
intensive_cases= regione_new.sort_values(by = ['IntensiveCarePatients'], ascending=False)
recovered= regione_new.sort_values(by = ['Recovered'], ascending=False)
home_confinement = regione_new.sort_values(by = ['HomeConfinement'], ascending=False)
deaths= regione_new.sort_values(by = ['Deaths'], ascending=False)

In [ ]:
#settaggio paerametri per il plot
sns.set(style = "darkgrid")

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(pop.Regione, pop.Popolazione, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(positive_cases.RegionName, positive_cases.CurrentPositiveCases, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(intensive_cases.RegionName, intensive_cases.IntensiveCarePatients, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(recovered.RegionName, recovered.Recovered, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(home_confinement.RegionName, home_confinement.HomeConfinement, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(deaths.RegionName, deaths.Deaths, color='b')
plt.show()

I grafici eseguiti sulle variabili ordinarie confermano quanto osservato nella tabella precedente di 'background'

A supporto delle osservazioni proposte per le features derivate, si esegue la rappresentazione grafica dei valori di queste.

In [ ]:
# Risultati in %
regione_new2['proportion_IC_HP_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['proportion_HC_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['proportion_CPC_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['proportion_R_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati di Letalità in %
regione_new2['proportion_D_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['ratio_IC_HP'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['ratio_TPC_TeP'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati di Prevalenza x 10000 abitanti
regione_new2['ratio_TPC_Pop'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati dei Posti di Terapia Intensiva x 10000
regione_new2['ratio_PIR_Pop'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati di Occupazione in %
regione_new2['proportion_IC_PIR'].sort_values().plot.barh(figsize=(10,10))

Le raffigurazioni sugli indici evidenziano sì, la prevalenza della Lombardia nelle prime posizioni, ma mostrano come a seconda dell'indice considerato possono variare le regioni nelle successive posizioni. 
Sembra che le regioni del Nord siano le più interessate, concordemente al fatto che l'epidemia si è sviluppata proprio in quelle zone. Tuttavia appare non scontato il fatto che la regione con più contagi sia quella effettivamente più colpita; si veda infatti il numero di posti in terapia intensiva rispetto ai pazienti effettivamente bisognosi di queste cure: la Valle d'Aosta e le Marche risultano analogamente interessata come la Lombardia, nonostante il loro numero di contagi sia decisamente inferiore. 

Di seguito vengono valutati gli andamenti di alcune features per regione. Si rimanda alla sezione relativa alle Time Series per approfondimenti.

In [ ]:
#Valutazione dell'anadamento dei pazienti ospedalizzati nell'arco di tempo presente nel dataset
fig, ax = plt.subplots(figsize=(20,10))
regione.groupby(['Date','RegionName'])['HospitalizedPatients'].max().unstack().plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
regione.groupby(['Date','RegionName'])['IntensiveCarePatients'].max().unstack().plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
regione.groupby(['Date','RegionName'])['TestsPerformed'].max().unstack().plot(ax=ax)

Il comportamento nel tempo degli attributi considerati evidenzia, in due casi, un andamento simil-esponenziale per le regioni, in modo particolare per la Lombardia e il Veneto(grafico n°3).
In un caso (grafico n°2) l'andamento sembra più proporzionale.
In tutte le raffigurazioni grafiche la Lombardia si presenta come la regione più interessata. 

Ora verranno considerati i valori NaN trovati all'inizio della trattazione.

In [ ]:
regione2

In [ ]:
pd.isnull(regione2)

In [ ]:
# I dati Nan sono ottenuti da una divisione per il valore zero. La sostituzione porta all'inserimento del valore 0 
regione3=regione2.fillna(0)
#datset regione3 è quello completo
regione3

In [ ]:
regione3.info()

Andiamo a valutare se sono presenti valori 'inf' e dove sono; qualora ci fossero questi verranno sostituiti con 0.


In [ ]:
df_attr1=regione3.iloc[:, 7:]
infi=np.isinf(df_attr1)
np.where(np.isinf(df_attr1))
df_attr1.index[np.isinf(df_attr1).any(1)]

In [ ]:
df_attr1.columns.to_series()[np.isinf(df_attr1).any()]

In [ ]:
print(regione3.iloc[47,21])
print(regione3.iloc[48,21])
print(regione3.iloc[367,22])

Dato l'esiguo numero di valori infiniti si procede con una sostituzione manuale.
Si procede poi con un controllo sull'effettiva sostituzione.

In [ ]:
regione3.iloc[47,21] = 0.0
regione3.iloc[48,21] = 0.0
regione3.iloc[367,22] = 0.0

In [ ]:
df_attr1=regione3.iloc[:, 7:]
infi=np.isinf(df_attr1)
np.where(np.isinf(df_attr1))
df_attr1.index[np.isinf(df_attr1).any(1)]

Si valuta se la correlazione tra le fatures, valutata nell'heatmap precedente solo sull'ultimo record cumulativo, è confermata per tutti i record del dataset.

In [ ]:
sns.set(font_scale=1.4)
plt.figure(figsize=(20,15))
sns.heatmap(regione3.iloc[:,7:].corr(),annot=True)
plt.axes().set_title('Heatmap n°2',fontsize =20)
plt.show()

Appare evidente la correlazione tra le features ordinarie e si evidenziano alcune correlazioni significative anche tra le features derivate.
Es: 'proportion_IC_PIR' e 'ratio_TCP_Pop_gg' = 86%

Osservando la sotto-mappa relativa alle features derivate si osserva che la correlazione più alta è proprio tra i due attributi citati nell'esempio qui sopra. Si tralasciano nella sotto-mappa in esame gli attributi con il 98%, in quanto strettamente dipendenti per la relazione matematica.
Da un'analisi retrospettiva emerge quindi che due indici importanti sono appunto 'proportion_IC_PIR' e 'ratio_TCP_Pop_gg', che possono essere impiegati per valutare le regioni più colpite dalla malattia. Appare quindi confermato quanto esposto in precedenza nella tabella di background.

---
## PCA <a name="pca"></a>

Si prosegue ora con l'esecuzione dell'analisi PCA, prima sulle features ordinarie e poi su quelle derivate.
Lo split dell'analisi in due parti serve per evitare una eccessiva richiesta di componenti principali per coprire la varianza totale.
La PCA con features ordinarie viene eseguita senza gli attributi 'Popolazione' e 'PostiIntensivaRegfime', perchè sono elementi costanti per ogni regione e andrebbero a fornire un identificativo uguale, per finalità, alla label.

In [ ]:
df_attr=regione3.iloc[:, 7:17]

In [ ]:
df_attr

Viene creata una variabile relativa a quelle che sono definibili delle label per ogni regione, cioè i codici regionali.

In [ ]:
df_reg=regione3.iloc[:, 3:4]

I dati vengono scalati con un MaxMin scaler, dato che dalla letteratura si evidenzia una robustezza dell'algoritmo nello scaling in un dato range.

In [ ]:
scaler = MinMaxScaler(feature_range=[0, 1])
data_rescaled = scaler.fit_transform(df_attr)

Si esegue la PCA al fine di valutare il numero di componenti necessarie per coprire la Varianza totale

In [ ]:
#Fitting dell'algoritmo di PCA con il dataset
pca = PCA().fit(data_rescaled)
#Plotting della somma cumulativa della Explained Variance
plt.figure(figsize=(10,10))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Dataset Explained Variance')
plt.show()

Dal grafico emerge che considerando 5 componenti si riesce a coprire tutta la varianza del dataset (circa 100%).
Al fine di eseguire anche un'analisi grafica si considerano 3 componenti principali.
Vengono eseguiti quindi i grafici 2D e 3D per la PCA, coprendo una Varianza totale > 98%

In [ ]:
pca = PCA(n_components=3, svd_solver='full')
principalComponents = pca.fit_transform(data_rescaled)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2',
                                                                  'principal component 3'])

Vengono concatenate le label alle componenti principali.

In [ ]:
finalDf = pd.concat([principalDf, df_reg], axis = 1)

In [ ]:
finalDf

In [ ]:
#2D PCA visulaization
fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(1,1,1)
#ax = plt.axes(projection='3d') 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
#ax.set_zlabel('Principal Component 3', fontsize = 20)
ax.set_title('2 component PCA', fontsize = 25)
targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
colors = ['red', 'green', 'blue', 'yellow','black', 'c', 'crimson','chocolate', 'maroon', 'purple', 'fuchsia', 'lime', 'olive', 
          'navy', 'teal', 'aqua','burlywood','chartreuse','orange', 'pink']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['RegionCode'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 20)
ax.legend(targets)
ax.grid()


In [ ]:
#3D PCA visualization
fig = plt.figure(figsize = (20,20))
ax = plt.axes(projection='3d') 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_zlabel('Principal Component 3', fontsize = 20)
ax.set_title('3 component PCA', fontsize = 25)
targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
colors = ['red', 'green', 'blue', 'yellow','black', 'c', 'crimson','chocolate', 'maroon', 'purple', 'fuchsia', 'lime', 'olive', 
          'navy', 'teal', 'aqua','burlywood','chartreuse','orange', 'pink']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['RegionCode'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , finalDf.loc[indicesToKeep, 'principal component 3']
               , c = color
               , s = 30)
ax.legend(targets)
ax.grid()


Dai grafici sembra che l'utilizzo di 2 e 3 componenti riesca a descrivere le classi relative alle regioni e a separarle adeguatamente.

Viene creato un dataframe contenente le componenti e le features esaminate.

In [ ]:
completeDf = pd.concat([principalDf, df_attr], axis = 1)

In [ ]:
completeDf = pd.concat([completeDf, df_reg], axis = 1)

In [ ]:
completeDf

Creazione del dataframe con le correlazioni tra fetaures e componenti principali e successiva rappresentazione grafica.

In [ ]:
corr_matrix = completeDf.corr()
corr_matrix

In [ ]:
fig = plt.figure(figsize=(15,15))

plt.imshow(completeDf.iloc[:, :-1].corr(), cmap = plt.cm.YlOrRd, interpolation='nearest')
plt.colorbar()
tick_marks = [i for i in range(len(completeDf.iloc[:, :-1].columns))]
plt.xticks(tick_marks, completeDf.iloc[:, :-1].columns, rotation='vertical')
plt.yticks(tick_marks, completeDf.iloc[:, :-1].columns)

In questo caso le features più correlate sono (escludendo le 'components', osservando l'asse orizzontale):
    
PC1 --> features più correlate 1, 2, 3, 5 e 9

PC2 --> features più correlate 7, 8 e 10

PC3 --> features più correlate 6, 7 e 8

Si confermano le correlazioni delle features evidenziate nell'heatmap n°1

Con il successivo comando visualizziamo in dettaglio le componenti della pca per meglio capire quali features sono rilevanti all'interno di ogni componente.

In [ ]:
print(abs( pca.components_ ))

pca.components_ ha la forma [n_components, n_features].

PC1 --> features più importanti 2,3,5 e 10

PC2 --> features più importanti 7 e 10

PC3 --> features più importanti 6,7 e 10

I passaggi eseguiti in precedenza per la PCA vengono riproposti, analizzando le sole features derivate.

In [ ]:
df_attr1=regione3.iloc[:, 19:]

In [ ]:
scaler1 = MinMaxScaler(feature_range=[0, 1])
data_rescaled1 = scaler1.fit_transform(df_attr1)

In [ ]:
#Fitting dell'algoritmo di PCA con il dataset
pca1 = PCA().fit(data_rescaled1)
#Plotting della somma cumulativa della Explained Variance
plt.figure(figsize=(10,10))
plt.plot(np.cumsum(pca1.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Dataset Explained Variance')
plt.show()

In [ ]:
pca1 = PCA(n_components=6)
principalComponents1 = pca1.fit_transform(data_rescaled1)
principalDf1 = pd.DataFrame(data = principalComponents1, columns = ['principal component 1', 'principal component 2', 
                                                                    'principal component 3', 'principal component 4', 
                                                                    'principal component 5','principal component 6'])

In [ ]:
finalDf1 = pd.concat([principalDf1, df_reg], axis = 1)

In [ ]:
#3D PCA visualization
fig = plt.figure(figsize = (20,20))
ax = plt.axes(projection='3d') 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_zlabel('Principal Component 3', fontsize = 20)
ax.set_title('3 component PCA', fontsize = 25)
targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
colors = ['red', 'green', 'blue', 'yellow','black', 'c', 'crimson','chocolate', 'maroon', 'purple', 'fuchsia', 'lime', 'olive', 
          'navy', 'teal', 'aqua','burlywood','chartreuse','orange', 'pink']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf1['RegionCode'] == target
    ax.scatter(finalDf1.loc[indicesToKeep, 'principal component 1']
               , finalDf1.loc[indicesToKeep, 'principal component 2']
               , finalDf1.loc[indicesToKeep, 'principal component 3']
               , c = color
               , s = 30)
ax.legend(targets)
ax.grid()

In [ ]:
completeDf1 = pd.concat([principalDf1, df_attr1], axis = 1)
completeDf1 = pd.concat([completeDf1, df_reg], axis = 1)

In [ ]:
corr_matrix1 = completeDf1.corr()
corr_matrix1

In [ ]:
fig = plt.figure(figsize=(15,15))

plt.imshow(completeDf1.iloc[:, :-1].corr(), cmap = plt.cm.YlOrRd, interpolation='nearest')
plt.colorbar()
tick_marks = [i for i in range(len(completeDf1.iloc[:, :-1].columns))]
plt.xticks(tick_marks, completeDf1.iloc[:, :-1].columns, rotation='vertical')
plt.yticks(tick_marks, completeDf1.iloc[:, :-1].columns)

L'analisi PCA sulle features derivate mette in evidenza il fatto che le features ordinarie sembrano più adatte ad ottenere PC meglio correlate. 
Si evidenziano conferme alle correlazioni tra le varie features derivate, riscontrate nell'heatmap n°2.

Si traccia l'andamento dei Test per l'Italia.

In [ ]:

data=regione2.groupby("Date")[['TotalPositiveCases', 'Deaths', 'Recovered','TestsPerformed','HospitalizedPatients',
                          'TotalHospitalizedPatients']].sum().reset_index()

In [ ]:
data

In [ ]:
def plot_df(data, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.xticks(rotation = 90)
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(data, x=data.Date, y=data.TestsPerformed, title='')

I dati sono stati raggruppati per regione; la funzione ordina per data e sposta i dati in ciascun gruppo. Infine, si applica una concatenazione per ottenere il dataframe nel suo formato originale.

In [ ]:
reg_group=regione3.groupby(["RegionName"])

In [ ]:
def lag_by_group(key, value_df):
    df = value_df.assign(group = key) 
    return (df.sort_values(by=["Date"], ascending=True)
        .set_index(["Date"])
        .shift(1)
               )

In [ ]:
dflist = [lag_by_group(g, reg_group.get_group(g)) for g in reg_group.groups.keys()]
pd.concat(dflist, axis=0).reset_index()

---
## LAG <a name="lag"></a>

I grafici seguenti sono relativi ad alcune features ordinarie, prima per l'Italia, poi per ogni regione, considerando 4 lag.


In [ ]:
reg_list=regione3['RegionName'].unique()
#print(reg_list[0])

In [ ]:
from pandas.plotting import lag_plot
plt.rcParams.update({'ytick.left' : False, 'axes.titlepad':10})


#names = ['HospitalizedPatients', 'Deaths', 'IntensiveCarePatients', 'Recovered', 'HomeConfinement','TotalPositiveCases',
      #   'TestsPerformed', 'ratio_PIR_Pop', 'ratio_IC_HP', 'ratio_TPC_TeP', 'ratio_TCP_Pop_gg', 'proportion_HC_TPC',
       #  'proportion_CPC_TPC','proportion_R_TPC', 'proportion_IC_PIR','proportion_D_TPC']

names2=['TotalPositiveCases', 'Deaths', 'Recovered','TestsPerformed','HospitalizedPatients', 'TotalHospitalizedPatients']

#for r in reg_list:
 #   df_pannel = regione3.loc[regione3.RegionName==r, :]
for n in names2:
    fig, axes = plt.subplots(1, 4, figsize=(11,5), sharex=True, sharey=True, dpi=100)
    for i, ax in enumerate(axes.flatten()[:4]):
        lag_plot(data[n], lag=i+1, ax=ax, c='firebrick')
        ax.set_title('Lag ' + str(i+1))
        


Le features selezionate mostrano che c'è correlazione, a livello nazionale, dato che non ho sparsità nelle occorrenze.

Seguono i grafici dei lag relativi alle features per ogni regione. 

In [ ]:
names = ['HospitalizedPatients', 'Deaths', 'IntensiveCarePatients', 'Recovered', 'HomeConfinement','TotalPositiveCases',
         'TestsPerformed']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names:
        fig, axes = plt.subplots(1, 4, figsize=(11,5), sharex=True, sharey=True, dpi=100)
        for i, ax in enumerate(axes.flatten()[:4]):
            lag_plot(df_pannel[n], lag=i+1, ax=ax, c='firebrick')
            ax.set_title('Lag ' + str(i+1))
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()

Per le features originali si evidenzia una correlazione tra le osservazioni con e senza lag. 

In [ ]:
names3 = ['ratio_IC_HP', 'ratio_TPC_TeP', 'ratio_TCP_Pop_gg', 'proportion_HC_TPC', 
          'proportion_CPC_TPC','proportion_R_TPC', 'proportion_IC_PIR','proportion_D_TPC']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names3:
        fig, axes = plt.subplots(1, 4, figsize=(11,5), sharex=True, sharey=True, dpi=100)
        for i, ax in enumerate(axes.flatten()[:4]):
            lag_plot(df_pannel[n], lag=i+1, ax=ax, c='firebrick')
            ax.set_title('Lag ' + str(i+1))
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()

Si evidenzia che è presente una maggior correlazione nelle osservazioni con lag per le features derivate relative a:  ratio_TCP_Pop_gg, proportion_IC_PIR, ratio_TPC_TeP

La forma del diagramma di lag fornisce indizi sulla struttura sottostante dei dati. Se è presente una forma lineare o si ha una forma tendente al lineare, questo suggerisce che probabilmente un modello di autoregressione è una scelta corretta.

---
## Autocorrelation e Partial Autocorrelation <a name="autocorrelation"></a>

Di seguito vengono riportati i grafici di autocorrelazione e autocorrelazione parziale.

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

names4 = ['HospitalizedPatients', 'IntensiveCarePatients', 'HomeConfinement','TotalPositiveCases', 'TestsPerformed']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names4:
        fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
        plot_acf(df_pannel[n].tolist(), lags=20, ax=axes[0])
        plot_pacf(df_pannel[n].tolist(), lags=20, ax=axes[1])
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()


I grafici dei valori per l'ACF rientrano nell'intervallo di confidenza del 95% per lag> 2, rappresentato dalla banda azzurra, il che evidenzia la mancanza di autocorrelazione tra le osservazioni, di ogni feature analizzata, al di sopra di quel valore di lag. 
L'andamento di ogni feature sembra simile per ogni regione.

In [ ]:
names5 = ['ratio_IC_HP', 'ratio_TPC_TeP', 'ratio_TCP_Pop_gg', 'proportion_HC_TPC','proportion_CPC_TPC', 'proportion_IC_PIR']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names5:
        fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
        plot_acf(df_pannel[n].tolist(), lags=20, ax=axes[0])
        plot_pacf(df_pannel[n].tolist(), lags=20, ax=axes[1])
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()

Analoghe considerazioni vengono effettuate per le features derivate analizzate.
Gli andamenti questa volta sono variabili per regione nel campo della stessa feature considerata.
Anche in questo caso per lag>2 si osserva una mancanza di autocorrelazione tra le osservazioni delle features.

Dall'osservazione del diagramma dell'autocorrelazione parziale si può intuire quanti lag passati includere nell'equazione di previsione di un modello auto-regressivo (AR). Nella maggior parte dei casi per lag>1 si rientra nella banda di confidenza, quindi il modello AR sarà di tipo 1.

---
## Time Series <a name="ts"></a>

In questa sezione ci dedichiamo alle seguenti analisi relative alle time series:
1. Visualizzazione trend 
2. Visualizzazione seasonality
3. Test stazionarietà
---

Utilizziamo il dataframe completo regione3, aggiungendo alcune features che specificano il giorno della settimana e il numero di settimana dell'anno. Per questo lasciamo invariato il dataframe originale e lavoriamo su un dataset temporaneo chiamato ts_df.

In [ ]:
ts_df=regione3.copy()

In [ ]:
ts_df.head()

Aggiungiamo le colonne day_of_week e week_of_year

In [ ]:
ts_df['day_of_week'] = ts_df.Date.apply(lambda x: x.dayofweek)
ts_df['week_of_year'] = ts_df.Date.apply(lambda x: x.weekofyear)

Stampiamo le prime cinque righe del dataframe appena modificato per vedere che le nuove colonne siano state effettivamente aggiunte.

In [ ]:
ts_df.head()

---
**Visualizzazione Trend**
---


Iniziamo lo studio delle time series analizzandone i grafici e vedendo se esistono trend.

Come primo passo definiamo due funzioni utili per disegnare i grafici relativi agli andamenti delle time series.

In [ ]:
# Draw Plot
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
#Disegna i grafici di tutte le regioni in relazione ad una singola feature.
def plot_ts_regions(df, x_ax, y_ax, regionlist, frequency='dayly'):
    for r in range(0, len(regionlist)):
        temp1 = df.loc[df.RegionName==regionlist[r], :]
        plot_df(temp1, x=temp1[x_ax], y=temp1[y_ax], title=frequency+' trend of '+y_ax+' '+reg_list[r])

Iniziamo generando i grafici relativi alle principali features, per tutte le regioni. Daremo poi delle considerazioni finali sulle informazioni che possiamo dedurre dai grafici generati.

**New positive cases**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'NewPositiveCases', reg_list)

**Hospitalized Patients**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'HospitalizedPatients', reg_list)

**Deaths**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'Deaths', reg_list)

**Intensive Care patients**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'IntensiveCarePatients', reg_list)

**Recovered**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'Recovered', reg_list)

**Home confinement**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'HomeConfinement', reg_list)

**Total Positive cases**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'TotalPositiveCases', reg_list)

**Tests performed**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'TestsPerformed', reg_list)

**Ratio IC_HP**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'ratio_IC_HP', reg_list)

**Ration TPC_TeP**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'ratio_TPC_TeP', reg_list)

**Ratio TCP_Pop_gg**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'ratio_TCP_Pop_gg', reg_list)

**Proportion HC_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_HC_TPC', reg_list)

**Proportion CPC_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_CPC_TPC', reg_list)

**Proportion R_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_R_TPC', reg_list)

**Proportion IC_PIR**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_IC_PIR', reg_list)

**Proportion D_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_D_TPC', reg_list)

---
**Considerazioni finali sui trend**



* NewPositiveCases: si nota un trend in crescendo nelle regioni più colpite, nelle regioni meno colpite invece si alternano picchi positivi e negativi, mantenendo comunque un trend crescente.

* HospitalizedPatients: in questo caso il trend risulta crescente in tutte le regioni, con curve continue nelle regioni più colpite.

* Deaths: in generale il trend è crescente nelle regioni più colpite, come la Lombardia, mentre in quelle meno colpite si hanno periodi spesso costanti, seguiti da alcuni picchi. Da notare che l'unica regione con valore costante a 0 è la Basilicata.

* IntensiveCarePatients: trend crescente per tutte le regioni.

* Recovered: trend crescente in tutte le regioni, con una maggiore ripidità nelle regioni meno colpite.

* HomeConfinement: trend crescente in tutte le regioni.

* TotalPositiveCases: trend crescente in tutte le regioni.

* TestsPerformed: trend crescente in tutte le regioni. 

* ratio_IC_HP: non è identificabile un vero e proprio trend, solo la regione Veneto sembra avere un trend in decrescita.

* ratio_TPC_TeP: crescente per tutte le regioni

* ratio_TCP_Pop_gg: crescente per tutte le regioni

* proportion_HC_TPC: in generale è crescente per quasi tutte le regioni, eccetto il Piemonte che ha un trend decrescente.

* proportion_CPC_TPC: Decrescente per le regioni con il maggior numero di casi, come Lombardia, Piemonte, Veneto ed Emilia Romagna, mentre nelle restanti abbiamo trend stabili, quasi costanti.

* proportion_R_TPC: abbiamo la Basilicata costante a 0, mentre per le altre regioni abbiamo un trend in generale decrescente per le regioni meno colpite e crescente invece per quelle più colpite.

* proportion_IC_PIR: in generale crescente in tutte le regioni

* proportion_D_TPC: in generale crescente in tutte le regioni, ad eccezione della Basilicata, costante a 0.

---
Visualizzazione Stagionalità
---

Come nella sezione relativa alla visualizzazione dei trend, anche qui definiamo delle funzioni utili alla creazione dei grafici.

In [ ]:
def weekly_seasonality_plot(df, featureName, reg_list, weeks, colors, week_dict):
    for r in range(0, len(reg_list)):
        temp1 = df.loc[df.RegionName==reg_list[r], :]
        print(reg_list[r])
        plt.figure(figsize=(14,10), dpi= 100)
        for i, y in enumerate(weeks):
            if i >= 0:        
                plt.plot('day_of_week', featureName, data=temp1.loc[temp1.week_of_year==y, :], color=colors[i], label=y)
                plt.text(temp1.loc[temp1.week_of_year==y, :].shape[0]-.9, temp1.loc[temp1.week_of_year==y, featureName][-1:].values[0], week_dict[y], fontsize=12, color=colors[i])
        plt.gca().set(title=featureName+' seasonality check', xlabel='day', ylabel=featureName)
        plt.show()

In [ ]:
weeks = ts_df['week_of_year'].unique()
# Prep Colors
np.random.seed(100)
mycolors = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(weeks), replace=False)
week_dict={9:'1st week',10:'2nd week',11:'3rd week',12:'4th week'}

Anche qui, come nella sezione precedente, generiamo prima tutti i grafici, e poi ne analizziamo i risultati in seguito nelle considerazioni finali sulla stagionalità.

**Legenda**


**Settimane**

* 1st week->prima settimana dall'inizio del contagio in Italia
* 2nd week->seconda settimana dall'inizio del contagio in Italia
* 3rd week->terza settimana dall'inizio del contagio in Italia
* 4th week->quarta settimana dall'inizio del contagio in Italia


**Giorni**

* 0: Lunedì
* 1: Martedì
* 2: Mercoledì
* 3: Giovedì
* 4: Venerdì
* 5: Sabato
* 6: Domenica

**New Positive cases**

In [ ]:
weekly_seasonality_plot(ts_df, 'NewPositiveCases', reg_list, weeks, mycolors, week_dict)

**Hospitalized Patients**

In [ ]:
weekly_seasonality_plot(ts_df, 'HospitalizedPatients', reg_list, weeks, mycolors, week_dict)

**Deaths**

In [ ]:
weekly_seasonality_plot(ts_df, 'Deaths', reg_list, weeks, mycolors, week_dict)

**Intensive care patients**

In [ ]:
weekly_seasonality_plot(ts_df, 'IntensiveCarePatients', reg_list, weeks, mycolors, week_dict)

**Recovered**

In [ ]:
weekly_seasonality_plot(ts_df, 'Recovered', reg_list, weeks, mycolors, week_dict)

**Home confinement**

In [ ]:
weekly_seasonality_plot(ts_df, 'HomeConfinement', reg_list, weeks, mycolors, week_dict)

**Total positive cases**

In [ ]:
weekly_seasonality_plot(ts_df, 'TotalPositiveCases', reg_list, weeks, mycolors, week_dict)

**Tests performed**

In [ ]:
weekly_seasonality_plot(ts_df, 'TestsPerformed', reg_list, weeks, mycolors, week_dict)

**Ratio IC_HP**

In [ ]:
weekly_seasonality_plot(ts_df, 'ratio_IC_HP', reg_list, weeks, mycolors, week_dict)

**ratio TPC_TeP**

In [ ]:
weekly_seasonality_plot(ts_df, 'ratio_TPC_TeP', reg_list, weeks, mycolors, week_dict)

**Ratio TCP_Pop_gg**

In [ ]:
weekly_seasonality_plot(ts_df, 'ratio_TCP_Pop_gg', reg_list, weeks, mycolors, week_dict)

**Proportion HC_TPC**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_HC_TPC', reg_list, weeks, mycolors, week_dict)

**Proportion CPC_TPC**

In [ ]:
weekly_seasonality_plot(ts_df,'proportion_CPC_TPC', reg_list, weeks, mycolors, week_dict)

**Proportion R_TPC**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_R_TPC', reg_list, weeks, mycolors, week_dict)

**Proportion IC_PIR**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_IC_PIR', reg_list, weeks, mycolors, week_dict)

**Proportion D_TPC**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_D_TPC', reg_list, weeks, mycolors, week_dict)

**Considerazioni finali sulla stagionalità**

Per quanto riguarda la stagionalità, in generale è difficile affermare che sia presente in qualcuna delle features considerate, soprattutto per la mancanza di uno storico di dati ed il breve periodo che ricoprono i dati disponibili. Per provare a vedere se esiste una stagionalità, abbiamo considerato le settimane, provando a vedere se determinati giorni, ad esempio il sabato e la domenica, avessero andamenti comuni e accentuati, in positivo o in negativo. Per features ordinarie, che hanno trend esclusivamente crescenti, come ad esempio IntensiveCarePatients, si potrebbe notare un minimo di stagionalità, mentre per le feature derivate no. Sarebbe stato indicativo e utile trovare stagionalità nel numero di nuovi casi positivi, ad esempio se si fosse trovata una marcata stagionalità nei giorni del week-end, si poteva supporre che, siccome nel weekend le persone tendono ad uscire e a creare assembramenti, si ha un maggiore numero di nuovi positivi. Questo però non accade, difatti anche se avessimo trovato stagionalità, i tempi di incubazione del virus sono diversi per ogni persona, quindi incrementi stagionali del numero di positivi in un dato giorno della settimana potrebbero essere semplici coincidenze. Concludendo, per i dati attualmente disponibili secondo noi non ci sono prove a sufficienza per affermare che vi sia stagionalità.

---
Verifica stazionarietà
---

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries):
    """
    Check Stationariety of time series.
    Please use np.array or pd.series as Input with your TS data only
    """
    #Convert numpy array to pandas serie
    if type(timeseries) is np.ndarray:
        df_timeseries = pd.Series(timeseries) 
        
    try:
        #Determing rolling statistics
        rolmean = df_timeseries.rolling(window=12).mean()
        rolstd = df_timeseries.rolling(window=12).std()

        #Plot rolling statistics:
        orig = plt.plot(timeseries, color='blue',label='Original')
        mean = plt.plot(rolmean, color='red', label='Rolling Mean')
        std = plt.plot(rolstd, color='black', label = 'Rolling Std')
        plt.legend(loc='best')
        plt.title('Rolling Mean & Standard Deviation')
        plt.show(block=False)

        #Perform Dickey-Fuller test:
        print('Results of Dickey-Fuller Test:')

        dftest = adfuller(timeseries, autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value
        
        # print(dfoutput)
    
        return dftest, dfoutput
    except Exception as message:
        print(f"Impossible to calc the stationariery of your TS: {message}")
        return None, None

In [ ]:
temp_stationary=ts_df.copy()
temp_stationary=temp_stationary[temp_stationary.RegionName == 'Lombardia']

In [ ]:
temp_stationary['Date']=pd.to_datetime(temp_stationary['Date'])

In [ ]:
x = temp_stationary['Date'].values
y1 = temp_stationary['NewPositiveCases'].values

In [ ]:
dftest, dfoutput = test_stationarity(y1)
dfoutput

In [ ]:
temp_stationary.columns

In [ ]:
x = temp_stationary['Date'].values
y1 = temp_stationary['IntensiveCarePatients'].values

In [ ]:
dftest, dfoutput = test_stationarity(y1)
dfoutput

---
Detrend
---

In [ ]:
#This is our TS
df_detrend = ts_df.copy()
df_detrend=df_detrend[df_detrend.RegionName=='Lombardia']
df_detrend.index = df_detrend.Date
df_detrend = df_detrend.drop('Date',axis=1)
plt.plot(df_detrend.index, df_detrend.IntensiveCarePatients)
plt.xticks(rotation=90)

In [ ]:
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
detrended = signal.detrend(df_detrend.IntensiveCarePatients.values)
plt.plot(df_detrend.index, detrended)
plt.xticks(rotation=90)
plt.title('IntensiveCarePatients by subtracting the least squares fit', fontsize=16)

---
## Clustering <a name="clustering"></a>

In questa sezione verranno eseguite delle procedure di clustering per valutare, sulla base di coppie di features selezionate, se le regioni del nord hanno un comportamento omogeneo ed uniforme oppure se le altre regioni italiane hanno adottato comportamenti simili alle regioni in cui il virus si è sviluppato maggiormente. 

Viene creato un nuovo dataframe contenente le feature da utilizzare per la clusterizzazione. 

In [ ]:
from sklearn.cluster import KMeans
regione_new.columns

Rimuoviamo le feature che non utilizzeremo

In [ ]:
data_k=regione_new.drop(['index', 'SNo', 'Date', 'Country', 'RegionCode', 'RegionName','Latitude', 'Longitude',
                         'ratio_PIR_Pop','ratio_IC_HP', 'ratio_TPC_TeP', 'ratio_TPC_Pop', 'ratio_D_Pop',
                         'proportion_IC_HP_TPC', 'proportion_HC_TPC','proportion_R_TPC','proportion_D_TPC'], axis=1)

In [ ]:
data_k
data_k.columns

In [ ]:
#Considero test eseguiti e casi totali
data_km=data_k.iloc[:,8:10]

Definiamo delle funzioni per il calcolo del wcss, ovvero per trovare il numero ottimale di cluster, e per la computazione e rappresentazione grafica del k-means e del dbscan.

In [ ]:

def calc_wcss(df):
    wcss=[]
    for i in range(1,11):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10)
        y_means = kmeans.fit(df)
        wcss.append(y_means.inertia_)
    #Plotting WCSS to find the number of clusters
    plt.plot(range(1,11), wcss)
    plt.xlabel("No. of clusters")
    plt.ylabel(" Within Cluster Sum of Squares")
    plt.show()
    return None

def kmeans_function(numberClusters, df, xlabel, ylabel ):
    # Fitting K-Means 
    kmeans = KMeans(n_clusters = numberClusters, init = 'k-means++', random_state = 42)
    y_kmeans = kmeans.fit_predict(df)
    #data_km3 = df
    #numerazione cluster partendo da 1
    y_kmeans4=y_kmeans
    y_kmeans4=y_kmeans+1
    # nuovo dataframe
    cluster = pd.DataFrame(y_kmeans4)
    # Aggiungo i cluster
    df['cluster'] = cluster
    #Media dei clusters
    kmeans_mean_cluster = pd.DataFrame(round(df.groupby('cluster').mean(),1))
    kmeans_mean_cluster
    #cluster dei positivi vs morti
    plt.figure(figsize=(8, 8))
    plt.scatter(df.iloc[:,0], df.iloc[:,1],c=y_kmeans, cmap='rainbow')  # plot points with cluster dependent colors
    plt.title('Clustering')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()
    data_r= pd.DataFrame()
    data_r['Region']=regione_new['RegionName']
    data_r['Level']=y_kmeans4
    for group in range(1,(numberClusters + 1)):
        re=data_r.loc[data_r['Level']==group]
        listofre= list(re['Region'])
        print("Group", group, ":", listofre)
    return y_kmeans4

def dbscan_function(df, regione_new, eps, min_samples):
    X = df
    X_scaled = scaler.fit_transform(X)
    #cluster dei dati
    dbscan = DBSCAN(eps=eps, min_samples = min_samples)
    clusters2 = dbscan.fit_predict(X_scaled)
    # plot dei cluster 
    plt.figure(figsize=(8, 8))
    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=clusters2, cmap="plasma")
    plt.xlabel("Feature 0")
    plt.ylabel("Feature 1")
    c2=pd.DataFrame(np.array(clusters2).T)
    cl_name=pd.concat([c2, regione_new['RegionName']], axis = 1)
    print('', cl_name)
    return clusters2

---
## K-Means <a name="kmeans"></a>

Andiamo a rappresentare graficamente la funzione wcss, within cluster sum of squares, per selezionare l'opportuno numero di cluster.

In [ ]:
calc_wcss(data_km)

Dal grafico della wcss si deduce che il numero ottimale di cluster è 5.

Eseguiamo la procedura di clustering considerando il numero di casi confermati rispetto al numero di test eseguiti.

In [ ]:
y_kmeans=kmeans_function(5, data_km, "No. of confirmed cases", "Test Perfomed by the Region")

Dall'analisi dei cluster si evidenzia che le regioni del nord, che sono le più interessate dal contagio, perchè sviluppatosi in quelle zone, rientrano in cluster diversi, pertanto il comportamento di alcune di queste risulta simile a quello delle altre regioni italiane meno interessate. Si osservi ad esempio il gruppo numero 5, nel quale compare il Lazio ed il Piemonte, e il gruppo 1. Si evidenzia anche la presenza di cluster ad un singolo elemento. 

Selezioniamo dal datafreme le seguenti feature: Frequenza dell'infezione per regione ed occupazione delle terapie intensive. 

In [ ]:
data_km1= pd.concat([data_k.iloc[:,12], data_k.iloc[:,14]], axis = 1)

Andiamo a rappresentare graficamente la funzione wcss, within cluster sum of squares, per selezionare l'opportuno numero di cluster.

In [ ]:
calc_wcss(data_km1)

Dal grafico della wcss si deduce che il numero opportuno di cluster è 4.

In [ ]:
y_kmeans1=kmeans_function(4, data_km1, "Frequencies of Infection by the Region", "Occupation IC")

Anche in questo contesto si riscontrano comportamenti simili nelle diverse regioni di Italia.

Selezioniamo come feature il numero di casi positivi per regione ed il numero di guariti.

In [ ]:
data_km2= pd.concat([data_k.iloc[:,4], data_k.iloc[:,6]], axis = 1)

Andiamo a rappresentare graficamente la funzione wcss, within cluster sum of squares, per selezionare l'opportuno numero di cluster.

In [ ]:
calc_wcss(data_km2)

Dal grafico sembra che sul 4 vi è una ulteriore lieve flessione, quindi poniamo come numero di cluster tale valore. 

In [ ]:
y_kmeans2=kmeans_function(4, data_km2, "Current Positive Cases by the Region", "Recovered")

Concordemente a quanto riscontrato sopra, si nota la presenza di cluster eterogenei (composti sia da regioni del nord, del centro e del sud)

Si procede con la creazione del nuovo dataframe, selezionando come feature il numero di casi positivi per la regione ed il numero di decessi. 

In [ ]:
data_km3= pd.concat([data_k.iloc[:,4], data_k.iloc[:,7]], axis = 1)

Andiamo a rappresentare graficamente la funzione wcss, within cluster sum of squares, per selezionare l'opportuno numero di cluster.

In [ ]:
calc_wcss(data_km3)

Dal grafico sembra che il numero ottimale di clusters sia 4.

In [ ]:
y_kmeans3=kmeans_function(4, data_km3,"Current Positive Cases by the Region", "Deaths")

Anche i questo caso notiamo che vi sono cluster eterogenei, comprendenti sia regioni del nord, che centro e sud. In particolare si nota che la Lombardia distanzia di molto tutte le altre regioni, formando un cluster unico, mentre i rimanenti cluster si concentrano in una porzione più piccola, con casistiche più simili. 

Creiamo un nuovo dataframe, considerando come features per la clusterizzazione il numero di casi positivi per regione e le frequenze di infezione.

In [ ]:
data_km4= pd.concat([data_k.iloc[:,4], data_k.iloc[:,12]], axis = 1)

Andiamo a rappresentare graficamente la funzione wcss, within cluster sum of squares, per selezionare l'opportuno numero di cluster.

In [ ]:
calc_wcss(data_km4)

Anche in questo caso il numero di cluster che sembra ottimale è 4.

In [ ]:
y_kmeans4=kmeans_function(4, data_km4, "Current Positive Cases by the Region", "Frequencies of Infection")

Si nota anche per questa clusterizzazione la presenza di gruppi di regioni eterogenee. Da notare inoltre che le regioni nei vari gruppi si distribuiscono lungo l'asse verticale, indicando quindi che anche regioni con numero di casi positivi simili hanno frequenze di infezioni diverse, quindi una situazione più o meno grave rispetto alle altre nello stesso gruppo. 

---
## DBSCAN <a name="dbscan"></a>

Proviamo un'altra tipologia di clustering, per controllare l'effettiva efficacia dei gruppi formati con il kmeans. Utilizzeremo gli stessi dataframe utilizzati precedentemente, ovvero i dataframe basati su coppie di feature. L'analisi viene effettuata seguendo lo stesso ordine utilizzato nel K-means.

In [ ]:
#DBSCAN
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
clusters=dbscan_function(data_km, regione_new, 0.1, 1)

In [ ]:
clusters1=dbscan_function(data_km1, regione_new, 0.4, 1)

In [ ]:
clusters2=dbscan_function(data_km2, regione_new, 0.3, 2)

In [ ]:
clusters3=dbscan_function(data_km3, regione_new, 0.3, 2)

In [ ]:
clusters4=dbscan_function(data_km4, regione_new, 0.3, 2)

Misuriamo le prestazioni di clustering tramite Rand Index: questo indice misura la similarità tra i cluster prodotti da due algoritmi. Dato che con K-means abbiamo valutato il giusto numero di cluster da creare (quindi il numero di cluster è noto), l'indice sopra citato valuterà la prestazione di DBSCAN rispetto a K_means.

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score

#Prestazioni DBSCAN rispetto a K-Means

#Clustering caso 1
print("ARI =", adjusted_rand_score(y_kmeans, clusters))
#Clustering caso 2
print("ARI =", adjusted_rand_score(y_kmeans1, clusters1))
#Clustering caso 3
print("ARI =", adjusted_rand_score(y_kmeans2, clusters2))
#Clustering caso 4
print("ARI =", adjusted_rand_score(y_kmeans3, clusters3))
#Clustering caso 5
print("ARI =", adjusted_rand_score(y_kmeans4, clusters4))


Dai risultati ottenuti nell'ARI possiamo affermare che le due tipologie di clustering performano in 3 casi su 5 perfettamente allo stesso modo.
Nel caso due si evidenzia una grande diversità, al contrario del caso 5 in cui la difformità è più ridotta. 

---
## Regressione <a name="regression"></a>

In [ ]:
data_m=regione3.drop(['SNo', 'Date', 'Country','RegionCode', 'RegionName','Latitude', 'Longitude','TotalHospitalizedPatients','CurrentPositiveCases',
                      'NewPositiveCases', 'Popolazione','TotalPositiveCases','ratio_PIR_Pop','ratio_IC_HP','TestsPerformed',
                      'ratio_TPC_TeP', 'ratio_TPC_Pop', 'ratio_D_Pop','proportion_IC_HP_TPC', 'proportion_HC_TPC','proportion_R_TPC',
                      'proportion_D_TPC','proportion_CPC_TPC'], axis=1)

In [ ]:
data_m

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor)
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import learning_curve
from sklearn import svm
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [ ]:
scaler=MinMaxScaler(feature_range=[0, 1])
data_m_sc=scaler.fit_transform(data_m)

In [ ]:
features_name=['ratio_TCP_Pop_gg','proportion_IC_PIR']
X_df = data_m.copy()
X_df.drop(['ratio_TCP_Pop_gg','proportion_IC_PIR'], axis=1, inplace=True)
y_df = data_m[features_name[0]].copy()

Si considera ora il precedente dataframe creato e si procede alla realizzazione del set di train e di test.
Si vogliono ottenere dei valori in output, corrispondenti di volta in volta alle varie features considerate nella variabile y, sulla base di determinate features scelte come input. 
La finalità è quella di avere un algoritmo che, dati dei valori in ingresso (es: totale contagiati, guariti, morti, ecc.) fornisca un valore il più possibile certo di frequenza di contagi e rapporto tra pazienti in terapia intensiva e posti disposnibili.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df,train_size=0.7, random_state=42, shuffle=True)

Vengono create delle strutture per permettere l'analisi del dataframe con più regressori e memorizzare i valori di RMSE e R^2, per ognuno di essi.
Le procedure descritte successivamente verranno ripetute per ciascun attributo 'ratio_TCP_Pop_gg', 'proportion_IC_PIR', da considerarsi come feature di output.

In [ ]:
Model0 = []
RMSE0 = [] #misura la differenza tra i valori
R_sq0 = [] #indica la corretteza del modello usato
cv0 = KFold(10, random_state = 42)

#funzione che, a seconda dell'algoritmo impigato, calcola i parametri RMSE e R_sq.
#Il nome dell'algoritmo viene inserito nella lista corrispondente, così come anche i parametri RMSE e R_sq.
def input_scores0(name, model, x, y):
    #accodo i nomi degli algoritmi
    Model0.append(name)
    #ricavo il valore meVengono create delle strutture per permettere l'analisi del dataframe con più regressori e memorizzare i valori di RMSE e R^2, per ognuno di essi.
    #eseguo poi la radice quadrata di tale valore medio per avere RMSE
    cv_result0=cross_val_score(model, x, y, cv=cv0, scoring='neg_mean_squared_error').mean()
    RMSE0.append(np.sqrt((-1) * cv_result0))
    #ricavo il valore medio del risultato prodotto dalla funzione di cross validatio (più valori del Coefficiente di determinazione)
    R_sq0.append(cross_val_score(model, x, y, cv=cv0, scoring='r2').mean())

Di seguito vengono inseriti i regressori esaminati, considerando in y, l'attributo 'ratio_TCP_Pop_gg'.

In [ ]:
names0 = ['Linear Regression', 'Ridge Regression', 'Lasso Regression', 'K Neighbors Regressor', 'Decision Tree Regressor', 
         'Random Forest Regressor', 'Gradient Boosting Regressor', 'Adaboost Regressor','Suppport Vector Machine']
models0 = [LinearRegression(), Ridge(), Lasso(), KNeighborsRegressor(), DecisionTreeRegressor(), RandomForestRegressor(),
          GradientBoostingRegressor(), AdaBoostRegressor(), SVR()]

for name0, model0 in zip(names0, models0):
    input_scores0(name0, model0, X_train, y_train)
    
#Costruisco un dataframe partendo da un dizionario
evaluation0 = pd.DataFrame({'Model0': Model0, 'RMSE0': RMSE0, 'R Squared0': R_sq0})
print("Valutazione del punteggio di training per il Dataset d'origine scalato: ")
evaluation0

Dalla tabella è possibile osservare che l'algoritmo più performante, usando una configurazione di default, è il Gradient boosting regressor.

Viene creato un nuovo set di training e test per eseguire l'analisi sul secondo attributo considerato come output, cioè proportion_IC_PIR, con conseguente inserimento dei regressori.

In [ ]:
y_df1 = data_m[features_name[1]].copy()
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_df, y_df1,train_size=0.7, random_state=42, shuffle=True)

Model1 = []
RMSE1 = [] #misura la differenza tra i valori
R_sq1 = [] #indica la corretteza del modello usato
cv1 = KFold(10, random_state = 42)

#funzione che, a seconda dell'algoritmo impigato, calcola i parametri RMSE e R_sq.
#Il nome dell'algoritmo viene inserito nella lista corrispondente, così come anche i parametri RMSE e R_sq.
def input_scores1(name, model, x, y):
    #accodo i nomi degli algoritmi
    Model1.append(name)
    #ricavo il valore medio del risultato ottenuto dalla funzione di cross validation (ho più valori medi negativi che poi moltiplico per -1)
    #eseguo poi la radice quadrata di tale valore medio per avere RMSE
    cv_result1=cross_val_score(model, x, y, cv=cv1, scoring='neg_mean_squared_error').mean()
    RMSE1.append(np.sqrt((-1) * cv_result1))
    #ricavo il valore medio del risultato prodotto dalla funzione di cross validatio (più valori del Coefficiente di determinazione)
    R_sq1.append(cross_val_score(model, x, y, cv=cv1, scoring='r2').mean())

In [ ]:
names1 = ['Linear Regression', 'Ridge Regression', 'Lasso Regression', 'K Neighbors Regressor', 'Decision Tree Regressor', 
         'Random Forest Regressor', 'Gradient Boosting Regressor', 'Adaboost Regressor','Suppport Vector Machine']
models1 = [LinearRegression(), Ridge(), Lasso(), KNeighborsRegressor(), DecisionTreeRegressor(), RandomForestRegressor(),
          GradientBoostingRegressor(), AdaBoostRegressor(), SVR()]

for name1, model1 in zip(names1, models1):
    input_scores1(name1, model1, X_train1, y_train1)
    
#Costruisco un dataframe partendo da un dizionario
evaluation1 = pd.DataFrame({'Model1': Model1, 'RMSE1': RMSE1, 'R Squared1': R_sq1})
print("Valutazione del punteggio di training per il Dataset d'origine scalato: ")
evaluation1

Anche nel secondo caso appare evidente che l'algoritmo migliore è il Gradient boosting regressor

Si considera ora l'ottimizzazione dei parametri tramite procedura di grid search, per entrambi i casi sopra considerati. 

In [ ]:
#Applicazione GridSearch su Gradient Boosting
#seleziono i parametri da stimare
param_grid = {'n_estimators':[10, 50, 100],
              'max_depth':[5, 10, 15],  
              'min_samples_split':[10, 15, 20], 
              'learning_rate':[0.001, 0.01, 0.1, 1]}

#imposto l'algoritmo di ricerca
clf = GridSearchCV(GradientBoostingRegressor(random_state=10), 
                   param_grid = param_grid, scoring='r2', 
                   cv=cv0).fit(X_train, y_train)

#stampo i parametri migliori dell'algoritmo
print(clf.best_estimator_) 
#stampo il mioglior valore per il parametro voluto
print("R Squared:",clf.best_score_)

In [ ]:
#Applicazione GridSearch su Gradient Boosting
#seleziono i parametri da stimare
param_grid1 = {'n_estimators':[10, 50, 600],
              'max_depth':[5, 10, 15],  
              'min_samples_split':[10, 15, 70], 
              'learning_rate':[0.001, 0.01, 0.1, 1]}

#imposto l'algoritmo di ricerca
clf1 = GridSearchCV(GradientBoostingRegressor(random_state=10), 
                   param_grid = param_grid1, scoring='r2', 
                   cv=cv1).fit(X_train1, y_train1)

#stampo i parametri migliori dell'algoritmo
print(clf1.best_estimator_) 
#stampo il mioglior valore per il parametro voluto
print("R Squared:",clf1.best_score_)

I valori ottenuti dopo la procedura di tuning sono conformi a quanto rilevato nelle tabelle precedenti.
Usiamo la seguente funzione per rappresentare successivamente dei grafici (https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html).

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(0.1, 1.0, 10)):
    
    plt.figure(figsize=(12,8))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    plt.legend(loc="best")
    
    return plt

Si inseriscono i valori dei parametri trovati nella fase di gridsearch

Si modficano manualmente i valori dei parametri per alcuni test (clf1_test) per migliorare i risultati

In [ ]:
clf_test = GradientBoostingRegressor(learning_rate=0.1, max_depth=5,
                                min_samples_split=10, n_estimators=100, 
                                random_state=10).fit(X_train, y_train)
print("Test RMSE: ", np.sqrt(mean_squared_error(y_test, clf_test.predict(X_test))))
print("Test R^2: ", r2_score(y_test, clf_test.predict(X_test)))
print("Score in train:", clf_test.score(X_train, y_train))
print("Score in test:", clf_test.score(X_test, y_test))
clf_test.feature_importances_

Si modficano manualmente i valori dei parametri per alcuni test (clf1_test) per migliorare i risultati

In [ ]:
clf1_test = GradientBoostingRegressor(learning_rate=0.1, max_depth=5,
                                min_samples_split=70, n_estimators=600, 
                                random_state=10).fit(X_train1, y_train1)
print("Test RMSE: ", np.sqrt(mean_squared_error(y_test1, clf1_test.predict(X_test1))))
print("Test R^2: ", r2_score(y_test1, clf1_test.predict(X_test1)))
print("Score in train:", clf1_test.score(X_train1, y_train1))
print("Score in test:", clf1_test.score(X_test1, y_test1))
clf1_test.feature_importances_

Come detto sopra, sono stati modificati i valori dei parametri riscontrati durante il processo di Grid search.
I parametri forniti dalla procedura di ricerca e definiti come i migliori per il training producevano overfitting nella fase di test. Pertanto modificando opportunamente i valori, tra quelli inseriti nel grid search, si è ottenuto un risultato senza overfitting, dato che la combinazione dei valori usata non risulta quella migliore e quindi possiede un valore di R^2 inferiore a quello identificato dalla procedura.


Come si può osservare in entrambe le prove i valori di test di R^2 sembrano essere decisamente buoni oltre ad aver ridotto il valori di RMSE, tuttavia rispetto ai valori di training si evidenza il fenomeno di overfitting.

Si considerano ora le curve di apprendimento in fase di training e di cross-validation e gli scatter plot true value vs predicted, per entrambi i casi.

In [ ]:
title = "Curve di Apprendimneto (Gradient Boosted Regression)" 
algorithm = GradientBoostingRegressor(learning_rate=0.1, max_depth=5,
                                       min_samples_split=10, n_estimators=100, 
                                       random_state=10)
cv=cv0
plot_learning_curve(algorithm, title, X_train, y_train, ylim=(0.5, 1.01), cv=cv)
plt.grid()
plt.show()

In [ ]:
## The line / model
history=algorithm.fit(X_train,y_train)
y_pred=algorithm.predict(X_test)
plt.scatter(y_test, y_pred)
x = np.linspace(0, 10, 1000)
plt.plot(x, x + 0, '-g')
plt.xlabel('True Values')
plt.ylabel('Predictions')

In [ ]:
title = "Curve di Apprendimneto (Gradient Boosted Regression)" 
algorithm1 = GradientBoostingRegressor(learning_rate=0.1, max_depth=15,
                                       min_samples_split=10, n_estimators=50, 
                                       random_state=10)
cv=cv1
plot_learning_curve(algorithm1, title, X_train1, y_train1, ylim=(0.5, 1.01), cv=cv)
plt.grid()
plt.show()

In [ ]:
## The line / model
algorithm1.fit(X_train1,y_train1)
y_pred=algorithm1.predict(X_test1)
plt.scatter(y_test1, y_pred)
x = np.linspace(0, 100, 10000)
plt.plot(x, x + 0, '-g')
plt.xlabel('True Values')
plt.ylabel('Predictions')

Confrontando i risultati relativi a R^2, nei rispettivi casi, con le curve sopra riportate si osserva che il mancato utilizzo della cross validation, in fase di training, porta ad un overfitting (confronto tra lo Score in test e l'ultimo valore della curva rossa), al contrario si osserva che lo score è superiore in fase di test (confronto Score in test con ultimo valore della curva verde).  

Per quanto riguarda gli scatter plot, si nota come il primo caso performi meglio del secondo, soprattutto verso valori più grandi. In generale però si osserva che i modelli effettuano buone previsioni, con la maggioranza dei punti che si collona intorno alla retta di identità.

Proviamo ora a visualizzare l'ordine di importanza delle features per il primo modello

In [ ]:
importances = algorithm.feature_importances_ 
print(importances)
indices = np.argsort(importances)[::-1]

In [ ]:
print ("Covid19 - Top 6 Important Features\n") 
for f in range(6):
    print("%d. %s (%f)" % (f + 1, X_df.columns[indices[f]], importances[indices[f]])) 
    #Plot the feature importances of the forest 

In [ ]:
print ("Mean Feature Importance %.6f" %np.mean(importances))

Creiamo un nuovo dataframe rimuovendo le features meno importanti, addestriamo un nuovo regressore sul dataframe creato e visualizziamo le differenti curve e scatterplot come effettuato in precedenza

In [ ]:
X_trimmed= X_df.copy()
X_trimmed.drop(['HospitalizedPatients','IntensiveCarePatients', 'Recovered'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df,train_size=0.7, random_state=42, shuffle=True)

In [ ]:
estimator = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, min_samples_leaf=9)
estimator.fit(X_train, y_train) 
print ("R-squared for Train: %.2f" %estimator.score(X_train, y_train) )
print ("R-squared for Test: %.2f" %estimator.score(X_test, y_test) )
plot_learning_curve(estimator, title, X_train, y_train, cv=cv) 
plt.show() 

In [ ]:
## The line / model
y_pred=estimator.predict(X_test)
plt.scatter(y_test, y_pred)
x = np.linspace(0, 10, 1000)
plt.plot(x, x + 0, '-g')
plt.xlabel('True Values')
plt.ylabel('Predictions')

Anche eliminando le features meno importanti, si osserva che il modello performa bene. 

---
## Forecasting <a name="forecasting"></a>

In questa sezione proviamo ad effettuare il forecast sui valori di TotalPositiveCases e NewPositiveCases della regione maggiormente colpita, ovvero della Lombardia. In questa sezione proveremo a predire un arco temporale di 30 giorni, utilizzando una semplice tecnica di curve fitting, successivamente utilizzeremo tecniche più avanzate, come ad esempio i modelli ARIMA.

Creiamo un dataframe contenente solo i dati relativi alla Lombardia.

In [ ]:
temp_stationary=ts_df.copy()
temp_stationary=temp_stationary[temp_stationary.RegionName == 'Lombardia']

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt

In [ ]:
temp_stationary.head()

In [ ]:
from scipy import optimize

Impostiamo come indice del dataframe la Data

In [ ]:
dtf=temp_stationary.copy()
## convert index to datetime
dtf.index = pd.to_datetime(dtf.Date, infer_datetime_format=True)

In [ ]:
dtf.head()

Definiamo tre possibili funzioni, una funzione lineare, una esponenziale ed una gaussiana

In [ ]:
'''
Linear function: f(x) = a + b*x
'''
def f(x):
    return 10 + 1500*x

y_linear = f(x=np.arange(len(dtf)))
'''
Exponential function: f(x) = a + b^x
'''
def f(x):
    return 10 + 1.18**x

y_exponential = f(x=np.arange(len(dtf)))
'''
Logistic function: f(x) = a / (1 + e^(-b*(x-c)))
'''
def f(x): 
    return 90000 / (1 + np.exp(-0.5*(x-20)))

y_logistic = f(x=np.arange(len(dtf)))

Visualizziamo in un unico grafico le tre funzioni sopra definite rispetto all'attuale andamento del totale dei positivi

In [ ]:
fig, ax = plt.subplots(figsize=(13,5))
ax.scatter(dtf["TotalPositiveCases"].index, dtf["TotalPositiveCases"].values, color="black")
ax.plot(dtf["TotalPositiveCases"].index, y_linear, label="linear", color="red")
ax.plot(dtf["TotalPositiveCases"].index, y_exponential, label="exponential", color="green")
ax.plot(dtf["TotalPositiveCases"].index, y_logistic, label="logistic", color="blue")
ax.legend()
plt.show()

Proviamo a verificare se per la feature Nuovi Casi positivi, l'andamento che ci aspettiamo è quello di una gaussiana

In [ ]:
'''
Gaussian function: f(x) = a * e^(-0.5 * ((x-μ)/σ)**2)
'''
def f(x):
    return 2000 * np.exp(-0.5 * ((x-24)/6)**2)

y_gaussian = f(x=np.arange(len(dtf)))

In [ ]:
fig, ax = plt.subplots(figsize=(13,5))
ax.bar(dtf["NewPositiveCases"].index, dtf["NewPositiveCases"].values, color="black")
ax.plot(dtf["NewPositiveCases"].index, y_gaussian, color="red")
plt.show()

Verificato che ci aspettiamo un andamento logistico per quanto riguarda il totale dei positivi e gaussiano per quanto riguarda il numero di nuovi positivi, procediamo con la definizione delle rispettive funzioni.

In [ ]:
'''
Logistic function: f(x) = capacity / (1 + e^-k*(x - midpoint) )
'''
def logistic_f(X, c, k, m):
    y = c / (1 + np.exp(-k*(X-m)))
    return y
## optimize from scipy
logistic_model, cov = optimize.curve_fit(logistic_f,
                                np.arange(len(dtf["TotalPositiveCases"])), 
                                dtf["TotalPositiveCases"].values, 
                                maxfev=10000,
                                p0=[np.max(dtf["TotalPositiveCases"]), 1, 1])
## print the parameters
logistic_model

In [ ]:
'''
Gaussian function: f(x) = a * e^(-0.5 * ((x-μ)/σ)**2)
'''
def gaussian_f(X, a, b, c):
    y = a * np.exp(-0.5 * ((X-b)/c)**2)
    return y
## optimize from scipy
gaussian_model, cov = optimize.curve_fit(gaussian_f,
                                np.arange(len(dtf["NewPositiveCases"])), 
                                dtf["NewPositiveCases"].values, 
                                maxfev=10000,
                                p0=[1, np.mean(dtf["NewPositiveCases"]), 1])
## print the parameters
gaussian_model

In [ ]:
'''
Function to fit. In this case logistic function:
    f(x) = capacity / (1 + e^-k*(x - midpoint) )
'''
def f(X, c, k, m):
    y = c / (1 + np.exp(-k*(X-m)))
    return y

Utilizziamo le seguenti funzioni per definire il curve fitting ed effettuare il forecast seguendo l'andamento delle curve.

In [ ]:
'''
Generate dates to index predictions.
:parameter
    :param start: str - "yyyy-mm-dd"
    :param end: str - "yyyy-mm-dd"
    :param n: num - length of index
    :param freq: None or str - 'B' business day, 'D' daily, 'W' weekly, 'M' monthly, 'A' annual, 'Q' quarterly
'''
def utils_generate_indexdate(start, end=None, n=None, freq="D"):
    if end is not None:
        index = pd.date_range(start=start, end=end, freq=freq)
    else:
        index = pd.date_range(start=start, periods=n, freq=freq)
    index = index[1:]
    print("--- generating index date --> start:", index[0], "| end:", index[-1], "| len:", len(index), "---")
    return index
'''
Fits a custom function.
:parameter
    :param X: array
    :param y: array
    :param f: function to fit (ex. logistic: f(X) = capacity / (1 + np.exp(-k*(X - midpoint)))
                                or gaussian: f(X) = a * np.exp(-0.5 * ((X-mu)/sigma)**2)   )
    :param kind: str - "logistic", "gaussian" or None
    :param p0: array or list of initial parameters (ex. for logistic p0=[np.max(ts), 1, 1])
:return
    optimal params
'''
def fit_curve(X, y, f=None, kind=None, p0=None):
    ## define f(x) if not specified
    if f is None:
        if kind == "logistic":
            f = lambda p,X: p[0] / (1 + np.exp(-p[1]*(X-p[2])))
        elif find == "gaussian":
            f = lambda p,X: p[0] * np.exp(-0.5 * ((X-p[1])/p[2])**2)
    
    ## find optimal parameters
    model, cov = optimize.curve_fit(f, X, y, maxfev=10000, p0=p0)
    return model
    


'''
Predict with optimal parameters.
'''
def utils_predict_curve(model, f, X):
    fitted = f(X, model[0], model[1], model[2])
    return fitted



'''
Plot parametric fitting.
'''
def utils_plot_parametric(dtf, zoom=30, figsize=(15,5)):
    ## interval
    dtf["residuals"] = dtf["ts"] - dtf["model"]
    dtf["conf_int_low"] = dtf["forecast"] - 1.96*dtf["residuals"].std()
    dtf["conf_int_up"] = dtf["forecast"] + 1.96*dtf["residuals"].std()
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=figsize)
    
    ## entire series
    dtf["ts"].plot(marker=".", linestyle='None', ax=ax[0], title="Parametric Fitting", color="black")
    dtf["model"].plot(ax=ax[0], color="green", label="model", legend=True)
    dtf["forecast"].plot(ax=ax[0], grid=True, color="red", label="forecast", legend=True)
    ax[0].fill_between(x=dtf.index, y1=dtf['conf_int_low'], y2=dtf['conf_int_up'], color='b', alpha=0.3)
   
    ## focus on last
    first_idx = dtf[pd.notnull(dtf["forecast"])].index[0]
    first_loc = dtf.index.tolist().index(first_idx)
    zoom_idx = dtf.index[first_loc-zoom]
    dtf.loc[zoom_idx:]["ts"].plot(marker=".", linestyle='None', ax=ax[1], color="black", 
                                  title="Zoom on the last "+str(zoom)+" observations")
    dtf.loc[zoom_idx:]["model"].plot(ax=ax[1], color="green")
    dtf.loc[zoom_idx:]["forecast"].plot(ax=ax[1], grid=True, color="red")
    ax[1].fill_between(x=dtf.loc[zoom_idx:].index, y1=dtf.loc[zoom_idx:]['conf_int_low'], 
                       y2=dtf.loc[zoom_idx:]['conf_int_up'], color='b', alpha=0.3)
    plt.show()
    return dtf[["ts","model","residuals","conf_int_low","forecast","conf_int_up"]]



'''
Forecast unknown future.
:parameter
    :param ts: pandas series
    :param f: function
    :param model: list of optim params
    :param pred_ahead: number of observations to forecast (ex. pred_ahead=30)
    :param end: string - date to forecast (ex. end="2016-12-31")
    :param freq: None or str - 'B' business day, 'D' daily, 'W' weekly, 'M' monthly, 'A' annual, 'Q' quarterly
    :param zoom: for plotting
'''
def forecast_curve(ts, f, model, pred_ahead=None, end=None, freq="D", zoom=30, figsize=(15,5)):
    ## fit
    fitted = utils_predict_curve(model, f, X=np.arange(len(ts)))
    dtf = ts.to_frame(name="ts")
    dtf["model"] = fitted
    
    ## index
    index = utils_generate_indexdate(start=ts.index[-1], end=end, n=pred_ahead, freq=freq)
    
    ## forecast
    preds = utils_predict_curve(model, f, X=np.arange(len(ts)+1, len(ts)+1+len(index)))
    dtf = dtf.append(pd.DataFrame(data=preds, index=index, columns=["forecast"]))
    
    ## plot
    utils_plot_parametric(dtf, zoom=zoom)
    return dtf

Effettuiamo il fitting ed il forecast basato sulla curva logistica per quanto riguarda il numero totale dei positivi in Lombardia.

In [ ]:
## Fit
model = fit_curve(X=np.arange(len(dtf["TotalPositiveCases"])), y=dtf["TotalPositiveCases"].values, f=f, p0=[np.max(dtf["TotalPositiveCases"]), 1, 1])
model

In [ ]:
## Forecast
preds = forecast_curve(dtf["TotalPositiveCases"], f, model, pred_ahead=30, freq="D", zoom=7, figsize=(15,5))

Effettuiamo il fitting ed il forecast per quanto riguarda il numero dei nuovi casi positivi in Lombardia, basandoci sulla funzione gaussiana.

In [ ]:
'''
Function to fit. In this case gaussian function:
    f(x) = a * e^(-0.5 * ((x-μ)/σ)**2)
'''
def f(X, a, b, c):
    y = a * np.exp(-0.5 * ((X-b)/c)**2)
    return y

In [ ]:
model = fit_curve(X=np.arange(len(dtf["NewPositiveCases"])), y=dtf["NewPositiveCases"].values, f=f, p0=[1, np.mean(dtf["NewPositiveCases"]), np.std(dtf["NewPositiveCases"])])
model

In [ ]:
## Forecast
preds = forecast_curve(dtf["NewPositiveCases"], f, model, pred_ahead=30, end=None, freq="D", zoom=7, figsize=(15,5))

---
## Modello Logistico <a name="logistic"></a>

In questa sezione verrà impiegato il modello logistico con lo scopo di valutare l'andamento dei contagi in Italia, al fine di definire un possibile periodo in cui ossevare una riduzione di questi ultimi.
Il modello logistico è ampiamente utilizzato per descrivere la crescita di una popolazione. Un'infezione può essere descritta come la crescita della popolazione di un agente patogeno, quindi l'uso di un modello logistico sembra ragionevole.

Si procede con la selezione del dataframe.

In [ ]:
data_tmp = data.copy()
data_tmp

Viene inserita una nuova feature relativa ai giorni trascorsi dall'inizio dell'anno.

In [ ]:
dayCount=data_tmp.Date.count()
ticks=range(0,dayCount,1)
data_tmp['DayNumber']=ticks
data_tmp['DayNumber']=data_tmp['DayNumber']+54
data_tmp.head()

Eseguiamo una previsione sul numero di contagiati con il modello logistico, selezionando gli attributi opportuni.

In [ ]:
data_temp = data_tmp.loc[:,['DayNumber','TotalPositiveCases']]
data_temp

Viene implementata la formula logistica come segue.

In [ ]:
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import datetime


def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))

In questa formula, abbiamo la variabile x che è il tempo e tre parametri:
- a si riferisce alla velocità dell'infezione
- b è il giorno in cui si è verificato il massimo numero di infezioni
- c è il numero totale di persone infette registrate alla fine dell'infezione

Per valori temporali alti, il numero di persone infette si avvicina sempre di più a c e questo è il punto in cui possiamo dire che l'infezione è terminata. Questa funzione ha anche un punto di flesso in b, che è il punto in cui la prima derivata inizia a diminuire (cioè il picco dopo il quale l'infezione inizia a diventare meno aggressiva e pertanto diminuisce).

Vengono stimati i valori dei parametri e gli errori a partire dai dati forniti.

In [ ]:
x = list(data_temp.iloc[:,0])
y = list(data_temp.iloc[:,1])

fit = curve_fit(logistic_model,x,y,p0=[2,100,20000])
fit

I due array risultanti corrispondono rispettivamente a: 
- i parametri a, b, c
- matrice della covarianza da cui estrarre gli errori
    
Gli errori vengono ottenuti tramite la radice quadrata dei valori posti sulla diaglonale della matrice.

In [ ]:
errors = [np.sqrt(fit[1][i][i]) for i in [0,1,2]]
errors

Gli errori ottenuti corrispondono agli errori standard dei parametri a, b e c, rispettivamente.

Vengono inseriti i parametri all'interno della funzione di risoluzione.

In [ ]:
a=fit[0][0]
b=fit[0][1]
c=fit[0][2]
sol = int(fsolve(lambda x : logistic_model(x,a,b,c) - int(c),b))

Si applica una modifica alla soluzione per ottenere una data relativa alla fine dei contagi.
La fine dell'infezione può essere calcolata come il giorno in cui il valore cumulativo delle persone infette è uguale al parametro c arrotondato al numero intero più vicino.

In [ ]:
datetime.datetime(2020, 1, 1)
dt = datetime.datetime(2020,1,1)
dtdelta = datetime.timedelta(days=sol)
d=dt + dtdelta
print('',d.strftime("%A"), '',d)

Si crea un grafico con l'andamento previsto rispetto a quello reale.

In [ ]:
pred_x = list(range(max(x),sol))
plt.rcParams['figure.figsize'] = [7, 7]
plt.rc('font', size=14)

In [ ]:
# Dati Reali
plt.scatter(x,y,label="Dati reali",color="red")
# Predizione della curva logistica
plt.plot(x+pred_x, [logistic_model(i,fit[0][0],fit[0][1],fit[0][2]) for i in x+pred_x], label="Modello Logistico" )

plt.legend()
plt.xlabel("Giorni dal 1 gennaio 2020")
plt.ylabel("Numero totale di persone infette")
plt.ylim((min(y)*0.9,c*1.1))
plt.show()

Si osserva che la curva logistica sembra approssimare il comportamento dei dati reali in maniera adeguata.
Tuttavia lo scopo di tale predizione è valutare un possibile periodo in cui potenzialmente si può avere una decrescita dei contagi.
Stando alla situazione attuale, il periodo riscontrato può essere plausibile in quanto, attualmente, sembra in atto la decrescita delle persone infette.

---
Modellazione Epidemiologica
---

In questa sezione si vuole eseguire un analisi al fine di modellizzare la diffusione della malattia, sfruttando modelli epidemiologici noti in letteratura.
I modelli considerati sono SIR e SEIR. Per quest'ultimo verrà analizzato anche il comportamento con diversi livelli di distanziamento sociale.

---
## Modello SIR <a name="sir"></a>

Il modello SIR è uno dei modelli compartimentali più semplici e molti modelli sono derivati da questa forma base.
Il modello si compone di tre compartimenti S,I,R, dove assumendo N costante, si ha N=I(t)+S(t)+R(t).
Ogni cmpartimento corrisponde a:
- S, suscettibili;
- I, infetti/infettivi;
- R, recuperati, guariti, non infettabili perché immuni, dopo aver contratto la malattia. Alcuni autori nei loro modelli interpretano la R, come resistenti o rimossi, in quanto non partecipano al processo epidemico, immuni o isolati o deceduti. Quindi vengono inclusi in R(t) anche i morti dovuti alla malattia.

L'evoluzione della malattia è generalmente formulata sotto forma di equazioni differenziali ordinarie (ODE).
I parametri in gioco, per questo modello, sono i seguenti:
- R con zero (riportato come 'r' nel codice)--> definito anche come numero di riproduzione, indica la potenziale trasmissibilità di una malattia infettiva. Più precisamente esso rappresenta il numero di nuovi casi generati, in media, da un singolo caso durante il proprio periodo infettivo, in una popolazione che altrimenti non sarebbe infetta. Questo parametro risulta relazionato con altri parametri tramite la seguente relazione: R=beta/gamma.
- beta --> corrisponde al tasso di contatto nella popolazione.
- gamma --> corrisponde all'inverso del periodo infettivo.

Per il primo parametro, il valore è stato ricavato da [1]

Per quanto riguarda gamma, il valore è stato prelevato da [2]

Si precisa che tutti i parametri risultano contestualizzati con i dati relativi al periodo in esame.

In [ ]:
from scipy.integrate import odeint

#R0
r=3.5
# Popolazione totale, N.
N = 60361700
# infetti all'inizio=I0 e recuperati=R0.
I0, R0 = (data_tmp['TotalPositiveCases'].min()-data_tmp['Deaths'].min()-data_tmp['Recovered'].min()), data_tmp['Deaths'].min()+data_tmp['Recovered'].min()
# Chiunque (S0) è suscettibile alla malattia, all'inizio.
S0 = N - I0 - R0
# beta: tasso di contatto nella popolazione sufficiente per la trasmissione della malattia
# gamma: inverso del tempo di infezione
beta, gamma = r*0.5, 0.5
# A grid of time points (in days)
t = np.linspace(0, 60, 60)

# modello SIR
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# vettore con le condizioni iniziali
y0 = S0, I0, R0
# Integrazione dell'equazione del modello nel tempo.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

# Stampo i grafici delle tre curve S(t), I(t) e R(t)

plt.figure(figsize=[10,10])
plt.grid()
plt.plot(t,S,label='Suscettibili')
plt.plot(t,I,label='Infetti')
plt.plot(t,R,label='Guariti/Recuperati')
plt.legend()
plt.xlabel('Tempo/giorni')
plt.ylabel('Numero soggetti')
plt.title('Modello SIR')
plt.grid()
plt.show()

Dal grafico si osserva che la naturale diffusione della malattia, senza alcuna forma di controllo, considerando anche il fatto di avere una propagazione più rapida rispetto a quello che può essere un adeguato rilevamento e intervento degli organi sanitari, comporta il contagio di circa un terzo della popolazione all'interno di un breve lasso temporale.
Tale considerazione risulta concorde con le ipotesi rilasciate nel periodo contestuale ai dati. 

---
## Modello SEIR <a name="seir"></a>

Per molte importanti infezioni c'è un significativo periodo di incubazione durante il quale gli individui sono stati infettati, ma non sono ancora infettivi. Durante questo periodo l'individuo si trova nel compartimento E che sta per "esposto". Supponendo che il periodo di incubazione sia una variabile casuale con distribuzione esponenziale con parametro alfa, considerando il periodo medio di incubazione come 1/alfa, con N costante, si ottiene il modello di seguito implementato.
Come per i precedenti parametri anche il valore di alfa compare nell'articolo sopra proposto.

Anche in questo caso si osserva l'impiego di equazioni diferenziali ordinarie.

In [ ]:
def base_seir_model(init_vals, params, t):
    S_0, E_0, I_0, R_0 = init_vals
    S, E, I, R = [S_0], [E_0], [I_0], [R_0]
    alpha, beta, gamma = params
    dt = t[1] - t[0]
    for _ in t[1:]:
        next_S = S[-1] - (beta*S[-1]*I[-1])*dt
        next_E = E[-1] + (beta*S[-1]*I[-1] - alpha*E[-1])*dt
        next_I = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
        next_R = R[-1] + (gamma*I[-1])*dt
        S.append(next_S)
        E.append(next_E)
        I.append(next_I)
        R.append(next_R)
    return np.stack([S, E, I, R]).T

Di seguito vengono definiti i parametri da applicare al modello, con successiva rappresentazione grafica.

In [ ]:
# Parameteri
t_max = 100
dt = 0.1
t = np.linspace(0, t_max, int(t_max/dt) + 1)
N = 60361700
init_vals = 1-((data_tmp['TotalPositiveCases'].min()-data_tmp['Deaths'].min()-data_tmp['Recovered'].min())/N), 0, (data_tmp['TotalPositiveCases'].min()-data_tmp['Deaths'].min()-data_tmp['Recovered'].min())/N, (data_tmp['Deaths'].min()+data_tmp['Recovered'].min())/N
alpha = 0.2
beta = 1.75 # ricordiamo r=beta/gamma con r=3,5
gamma = 0.5
params = alpha, beta, gamma

results = base_seir_model(init_vals, params, t)
S, E, I, R=results.T

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()
plt.plot(t,S,label='Suscettibili')
plt.plot(t,E,label='Esposti')
plt.plot(t,I,label='Infetti')
plt.plot(t,R,label='Guariti/Recuperati')
plt.legend()
plt.xlabel('Tempo (giorni)')
plt.ylabel('%Numero soggetti')
plt.title('Modello SEIR')
plt.grid()
plt.show()

Si osserva che in questo caso, senza distanziamento sociale o altra forma di prevenzione, al culmine, il 10% della popolazione risulta infettata dalla malattia dopo circa 50 giorni dalla prima esposizione, con alte probabilità di essere un'infezione grave.

Da quanto osservato nei grafici emerge che una percentuale rilevante della popolazione risulta essere potenzialmente colpita. Tali valori possono essere coerenti con il ritardo degli interventi di rilevamento, date le scarse informazioni note sulla malattia in quel periodo, ma soprattutto per il fatto che il virus è rimasto latente e sottotraccia per un lungo periodo di tempo. Quindi i dati noti rappresentano solo una parte di quella che era la situazione nel contesto analizzato.

---
## Modello SEIR con distanziamento sociale <a name="seird"></a>

Si considera ora il modello SEIR, cercando di modellare l'effetto del distanziamento o di altre forme di prevenzione.
Nel modello infatti si cerchera di agire sul parametro beta, introducento un parametro rho che tenga conto degli effetti della prevenzione. Rho è un termine costante tra 0 e 1, dove 0 indica un blocco totale di ogni attività, mentre 1 equivale al caso sopra esposto (con assenza di forme preventive). Si procede pertanto moltiplicando beta per il nuovo parametro inserito.

In [ ]:
def seir_model_with_soc_dist(init_vals, params, t):
    S_0, E_0, I_0, R_0 = init_vals
    S, E, I, R = [S_0], [E_0], [I_0], [R_0]
    alpha, beta, gamma, rho = params
    dt = t[1] - t[0]
    for _ in t[1:]:
        next_S = S[-1] - (rho*beta*S[-1]*I[-1])*dt
        next_E = E[-1] + (rho*beta*S[-1]*I[-1] - alpha*E[-1])*dt
        next_I = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
        next_R = R[-1] + (gamma*I[-1])*dt
        S.append(next_S)
        E.append(next_E)
        I.append(next_I)
        R.append(next_R)
    return np.stack([S, E, I, R]).T

Vengono eseguiti due valutazioni comparandole con il caso base.
I valori per rho sono quindi: 1, 0.5 e 0.35. Successivamente vengono rappresentate le curve degli infetti e degli esposti.

In [ ]:
t_m = 400
dt = 0.1
t1 = np.linspace(0, t_m, int(t_m/dt) + 1)

rho1= 1
rho2= 0.5
rho3 = 0.35

params1 = alpha, beta, gamma, rho1
results1 = seir_model_with_soc_dist(init_vals, params1, t1)
S1, E1, I1, R1=results1.T

params2 = alpha, beta, gamma, rho2
results2 = seir_model_with_soc_dist(init_vals, params2, t1)
S2, E2, I2, R2=results2.T

params3 = alpha, beta, gamma, rho3
results3 = seir_model_with_soc_dist(init_vals, params3, t1)
S3, E3, I3, R3=results3.T

In [ ]:
plt.figure(figsize=[10,10])
plt.grid()

#plt.plot(t1,S1,label='Suscettibili')
plt.plot(t1,E1,label='Esposti')
plt.plot(t1,I1,label='Infetti')
#plt.plot(t1,R1,label='Guariti')

#plt.plot(t1,S2,'--',label='Suscettibili')
plt.plot(t1,E2,'--',label='Esposti')
plt.plot(t1,I2,'y--',label='Infetti')
#plt.plot(t1,R2,'k--',label='Guariti')

#plt.plot(t1,S3,'c:',label='Suscettibili')
plt.plot(t1,E3,':',label='Esposti')
plt.plot(t1,I3,'m:',label='Infetti')
#plt.plot(t1,R3,':',label='Guariti/Recuperati')

def connectpoints(x,y,p1,p2,p3):
    x1, x2, x3 = x[p1], x[p2], x[p3]
    y1, y2, y3 = y[p1], y[p2], y[p3]
    plt.plot([x1,x2,x3],[y1,y2,y3],'k-')
    
time = [50, 123, 320]
perc = [0.10, 0.031, 0.005]

for i in range(0, len(time)):
    plt.plot(time[i], perc[i], 'ro-')

i=0
connectpoints(time,perc,i,i+1,i+2)

plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plt.xlabel('Tempo (giorni)')
plt.ylabel('%Numero soggetti')
plt.title('Modello SEIR')
plt.grid()
plt.show()

Si può notare un effetto di appiattimento nelle curve degli infetti e degli esposti, con l'aumentare delle restrizioni (quindi riducendo il parametro rho).
Le azioni di distanziamento e prevenzione sembrano mostrare un contenimento dei contagi nei primi periodi, spostando il picco degli infetti avanti nel tempo, permettendo la preparazione di pratiche di adeguamento sociale e sanitario. Considerando la successione dei picchi, con diversi valori di rho, si nota che la decrescita sembra esponenziale.
La presenza di picchi relativi agli infetti ed agli esposti, nelle curve relative al distanziamento, è dovuta al fatto che il virus riduce la sua interazione con i soggetti, senza comunque essere eliminato, in quanto resta latente nell'ambiente.

L'analisi con distanziamento sociale mette pertanto in luce la possibilità di miglioramenti nell'ambito della sopravvivenza alla malattia, fornendo più tempo per lo sviluppo di trattamenti e cure, con un mantenimento dei picchi più bassi.
Quanto detto sembra essere conforme a quanto accade oggi in Italia, dove si evidenzia un calo degli infetti grazie al periodo di lockdown.

---
## SARIMAX <a name="arima"></a>

Proviamo ad effettuare forecast più precisi rispetto al semplice curve fitting, utilizzando il metodo SARIMAX. In questo caso creiamo un dataframe specifico, composto dall'indice che sarà la data e dalla feature di cui intediamo effettuare la predizione. Il dataframe creato possiede come feature il numero totale di positivi a livello italiano.

In [ ]:
def split_data(data, split_date):
    return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()

def limit(data, frm, to):
    return data[(data.index>=frm)&(data.index<to)]

In [ ]:
t = data_tmp.copy()
t.set_index('Date', inplace=True) 
t.index = pd.to_datetime(t.index)
t.drop(['TotalHospitalizedPatients','DayNumber', 'Recovered', 'Deaths', 'TestsPerformed', 'HospitalizedPatients'], axis=1, inplace=True)
t.shape

Effettuiamo una decomposizione della serie per visualizzare i trend e la stagionalità, nonchè i residui.

In [ ]:
#### from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(t, model='additive', freq=7)
fig = result.plot()

Verifichiamo se la serie sia stazionaria o non stazionaria.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualisation
import matplotlib.pyplot as plt # data visualisation
import datetime as dt # working with time data
import warnings  
warnings.filterwarnings('ignore')

from statsmodels.tsa.stattools import adfuller

results = adfuller(t["TotalPositiveCases"])
print('ADF Statistic: %f' % results[0])
print('p-value: %f' % results[1])
print('Critical Values:')
for key, value in results[4].items():
    print('\t{}: {}'.format(key, value))

La serie sembra stazionaria, considerando ADF e p-value rispetto ai valori critici.

Visualizziamo l'autocorrelazione e l'autocorrelazione parziale

In [ ]:
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig,ax = plt.subplots(2,1,figsize=(20,10))
plot_acf(t, lags=27, ax=ax[0])
plot_pacf(t, lags=27, ax=ax[1])
plt.show()

In [ ]:
from pandas.plotting import lag_plot

fig, ax = plt.subplots(figsize=(10, 10))
ax = lag_plot(t, lag=1)
ax = lag_plot(t, lag=2, c="orange")

plt.show()

Tramite la visualizzazione delle varie metriche, come Akaike Information Critera (AIC) and Bayesian Information Criteria (BIC), proviamo a trovare la migliore combinazione di parametri per addestrare il modello.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

arima_df = pd.DataFrame(columns=["p","q","aic","bic"])

i=0
# Loop over p values from 0-3
for p in range(6):
    # Loop over q values from 0-3
    for q in range(6):
        
        try:
            # creating and fitting ARIMA(p,1,q) model
            model = ARIMA(t.astype(float), order=(p,1,q))
            results = model.fit()
            
            # Printing order, AIC and BIC
            #print(p, q, results.aic, results.bic)
            arima_df.loc[i,"p"] = p
            arima_df.loc[i,"q"] = q
            arima_df.loc[i,"aic"] = results.aic
            arima_df.loc[i,"bic"] = results.bic
            i = i+1
        except:
            #print(p, q, None, None)
            i = i+1
    
arima_df["sum_aic_bic"] = arima_df["aic"]+arima_df["bic"]
arima_df.sort_values(by="sum_aic_bic", ascending=False, inplace=True)
arima_df

Notiamo che la migliore combinazione è quella data da p=1 ed q=0, dal momento che è la combinazione che possiede il valore AIC minore.

Addestriamo il modello sarimax utilizzando la suddetta combinazione di parametri e ne stampiamo le varie metriche.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model2 = SARIMAX(t, order=(1,1,0), seasonal_order=(0,1,0,12))
results = model2.fit()
results.summary()

In [ ]:
plt.rcParams['figure.figsize'] = 12, 8
plot = results.plot_diagnostics()

Dalle metriche e dai plot effettuati notiamo che il modello è abbastanza affidabile, infatti ad esempio i quantili si distribuiscono bene intorno alla retta.

Effettuiamo il forecast del valore di TotalPositiveCases per i successivi 10 giorni rispetto all'ultima data presente nel nostro dataset, ne mostriamo i valori in una tabella riassuntiva ed infine ne mostriamo i grafici con i rispettivi intervalli di confidenza.

In [ ]:
# Create SARIMA mean forecast
forecast = results.get_forecast(steps=10)

In [ ]:
# Note: since we did not specify the alpha parameter, the
# confidence level is at the default, 95%
print(forecast.summary_frame())

In [ ]:
endog = t['TotalPositiveCases']
endog.plot(figsize=(15, 5))

In [ ]:
import statsmodels.api as sm
# Construct the model
mod = sm.tsa.SARIMAX(endog, order=(2, 1, 0))
# Estimate the parameters
res = mod.fit()

print(res.summary())

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

# Plot the data (here we are subsetting it to get a better look at the forecasts)
#endog.plot(ax=ax)

# Construct the forecasts
fcast = res.get_forecast(steps=10).summary_frame()
fcast['mean'].plot(ax=ax, style='k--')
ax.fill_between(fcast.index, fcast['mean_ci_lower'], fcast['mean_ci_upper'], color='k', alpha=0.1);

Notiamo come nei primi giorni di predizione l'intervallo di confidenza sia ridotto, ed aumenti man mano che si effettuano predizioni più avanti nel tempo

---
## Altri modelli per Forecasting <a name="otherforecasting"></a>

Proviamo ad effettuare il forecast, sempre sul numero totale di positivi a livello italiano, utilizzando ulteriori modelli, quali KernelRidge, SVR e MLPRegressor, mostrandone i grafici comparativi di predizioni e attuali.

In [ ]:
from sklearn.svm import SVR, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error

def tsplit(X,y,model):
    tscv = TimeSeriesSplit(n_splits=3)
    fig,ax = plt.subplots(3, figsize=(15,8))
    axis = 0
    for train_index, test_index in tscv.split(X):
        #splitting data into training and test sets
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]
        #fitting model
        model.fit(X_train,y_train.values.ravel())
        #predicting
        predictions = model.predict(X_test)
        #printing results
        print("MSE for split {0}:".format(axis+1))
        print(mean_squared_error(y_test,predictions))
        #ax[axis].plot(X_train.index, y_train) # needs fixing
        ax[axis].plot(list(X_test.index), predictions)
        ax[axis].plot(list(X_test.index), y_test)
        axis += 1
    return(None)

In [ ]:
t = data_tmp.copy()
t.drop(['TotalHospitalizedPatients','Date', 'Recovered', 'Deaths', 'TestsPerformed', 'HospitalizedPatients'], axis=1, inplace=True)
t.shape
X = t[["DayNumber"]]
y = t[["TotalPositiveCases"]]

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(t["DayNumber"],t["TotalPositiveCases"],c=t["DayNumber"])
plt.legend()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_y = StandardScaler()
# Scale x and y (two scale objects)
X2_scaled = pd.DataFrame(sc_x.fit_transform(X))
y2_scaled = pd.DataFrame(sc_y.fit_transform(y))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge

In [ ]:
rdg = KernelRidge(kernel='rbf')
parameters = {'alpha':np.arange(0.005,0.02,0.005), 'gamma':np.arange(0.001,0.01,0.001)}

tscv = TimeSeriesSplit(n_splits=3)
rdg_gs = GridSearchCV(rdg, parameters, cv=tscv, verbose=0, scoring='neg_mean_squared_error')
rdg_gs.fit(X2_scaled, y2_scaled)

rdg_gs.best_score_
best_rdg = rdg_gs.best_estimator_
print(best_rdg)

In [ ]:
tsplit(X2_scaled,y2_scaled,best_rdg)

In [ ]:
svr = SVR()
parameters = {'kernel':['rbf','poly'],
              'C':np.arange(0.2,0.8,0.1),
              'gamma':np.arange(0.2,1.2,0.02),
              'degree':[3,4,5]}

tscv = TimeSeriesSplit(n_splits=3)
reg = GridSearchCV(svr, parameters, cv=tscv, verbose=0, scoring='neg_mean_squared_error')
reg.fit(X2_scaled, y2_scaled.values.ravel())

reg.best_score_
best_svr = reg.best_estimator_
print(best_svr)

In [ ]:
tsplit(X2_scaled,y2_scaled,best_svr)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import TimeSeriesSplit

mlp = MLPRegressor(max_iter=600)
parameters = {'hidden_layer_sizes':np.arange(800,1400,50),'alpha':[0.0001,0.0002], 'momentum':[0.85,0.9,0.95]}

tscv = TimeSeriesSplit(n_splits=3)
reg = GridSearchCV(mlp, parameters, cv=tscv, verbose=0, scoring='neg_mean_squared_error')
reg.fit(X2_scaled, y2_scaled.values.ravel())

reg.best_score_
best_mlp = reg.best_estimator_
print(best_mlp)

In [ ]:
tsplit(X2_scaled,y2_scaled,best_mlp)

Dai grafici notiamo che l'unico modello che approssima con un buon grado i valori reali con quelli predetti è il MLPRegressor, mentre nel KernelRidge e nel SVR abbiamo risultati non convincenti.

---
## References <a name="references"></a>

[1] R0 modello sir: http://www.salute.gov.it/portale/news/p3_2_1_1_1.jsp?lingua=italiano&menu=notizie&p=dalministero&id=4429

[2] Gamma modello sir: https://www.medrxiv.org/content/10.1101/2020.04.17.20053157v1.full.pdf

[3] https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html
 